# 国土数値情報の取得

## Library Import

In [1]:
# データの取り扱いに関するライブラリ
import numpy as np # 高速計算
import pandas as pd # 表データの扱い

# 可視化に関するライブラリ
import matplotlib.pyplot as plt
import japanize_matplotlib

import geopandas as gpd
from sklearn.neighbors import NearestNeighbors

import gc
from pathlib import Path
import re
from pyogrio import read_dataframe

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 200)

In [2]:
# 自身がファイルを格納したディレクトリを指定
ROOT_DIR = '../input/'
train_file_path = ROOT_DIR + 'train.csv'
test_file_path = ROOT_DIR + 'test.csv'
intermediate_path = '../output/intermediate_file/'
gis_path = ROOT_DIR + 'GISデータ/'

pkey_cols = ['target_ym', 'building_id', 'unit_id']
geo_cols = ['lat', 'lon']

geo_ver = 2

## File Import

In [3]:
train_df_geo = pd.read_csv(train_file_path)[pkey_cols + geo_cols]
test_df_geo = pd.read_csv(test_file_path)[pkey_cols + geo_cols]

## データの変換

#### 対象年の抽出

In [4]:
def parse_year(date_input):
    try:
        s = str(date_input)
        if len(s) < 4:
            return np.nan
        return int(s[:4])
    except:
        return np.nan

In [5]:
train_df_geo['target_year'] = train_df_geo['target_ym'].apply(parse_year)
test_df_geo['target_year'] = test_df_geo['target_ym'].apply(parse_year)

#### 測地系の変換(EPSG:6668 → EPSG:4612)

In [6]:
def fill_jgd_from_wgs(df, lon_wgs_col='lon', lat_wgs_col='lat'):
    """
    JGD2011(EPSG:6668) → JGD2000(EPSG:4612) に変換して補完する
    """

    # 2) 世界測地系の GeoDataFrame
    gdf_wgs = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df[lon_wgs_col], df[lat_wgs_col]),
        crs='EPSG:6668'
    )

    # 3) 日本測地系（JGD2000, EPSG:4612）に変換
    gdf_jgd = gdf_wgs.to_crs(epsg=4612)

    # 4) 変換後の座標を el/nl に入れる
    df['lon_jgd'] = gdf_jgd.geometry.x.values  # 経度（4612）
    df['lat_jgd'] = gdf_jgd.geometry.y.values  # 緯度（4612）

    return df

In [7]:
train_df_geo = fill_jgd_from_wgs(train_df_geo)
test_df_geo = fill_jgd_from_wgs(test_df_geo)

## 地価

In [8]:
# land_gdf2022 = read_dataframe(f'{gis_path}公示地価/L01-22_GML/L01-22.geojson') 
land_gdf = read_dataframe(f'{gis_path}公示地価/L01-23_GML/L01-23.geojson') 
print(land_gdf.crs) # EPSG:4612(日本測地系2000)

EPSG:4612


In [9]:
land_gdf = land_gdf.rename(columns={
    'L01_005': 'land_year',            # 年度（例: 2023）
    'L01_006': 'land_price_2023',    # 公示地価 [円/㎡]
    'L01_007': 'land_price_yoy_pct',   # 対前年変動率 [%]
})

year_to_col = {y: f'L01_{y - 1922:03d}' for y in range(2018, 2023)}
rename_price_cols = {col: f'land_price_{year}' 
                     for year, col in year_to_col.items()}

land_gdf = land_gdf.rename(columns=rename_price_cols)

In [10]:
def add_land_price_features_haversine(
    train_df: pd.DataFrame,
    test_df: pd.DataFrame,
    land_gdf,
    lat_col: str,
    lon_col: str,
    year_col: str,
    n_neighbors: int = 3,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    target_year=y の各行について、標準地を (land_price_y>0 かつ land_price_{y-1}>0) に絞ってから
    最近傍/距離加重の地価特徴量と前年比(yoy/dlog)を作る。

    目的:
    - 「前年が存在しない新設標準地（前年=0）」を参照して train に 0 地価が入るのを避ける
    - yoy/dlog を意味のある標準地だけで計算する

    前提:
    - year_col は 2019 のような年
    - land_gdf に land_price_YYYY 列がある
    - lat/lon は経緯度（度）
    """

    if n_neighbors <= 0:
        raise ValueError('n_neighbors must be positive')

    # --- 1) train/test を MultiIndex で安全に結合（index重複回避） ---
    combined = pd.concat({'train': train_df, 'test': test_df}, axis=0, names=['__set__'])

    # --- 2) 有効座標のみ ---
    lat = combined[lat_col]
    lon = combined[lon_col]
    valid_mask = (
        lat.notna() & lon.notna()
        & lat.between(-90, 90)
        & lon.between(-180, 180)
    )
    df_valid = combined.loc[valid_mask]

    # --- 3) 出力（MultiIndexで持つ） ---
    nearest_all = pd.Series(np.nan, index=combined.index, dtype='float64')
    dist_all = pd.Series(np.nan, index=combined.index, dtype='float64')
    weighted_all = pd.Series(np.nan, index=combined.index, dtype='float64')
    nearest_prev_all = pd.Series(np.nan, index=combined.index, dtype='float64')
    weighted_prev_all = pd.Series(np.nan, index=combined.index, dtype='float64')

    if len(df_valid) > 0:
        # 物件座標（df_validの行順で固定）
        prop_lat_rad = np.deg2rad(df_valid[lat_col].to_numpy())
        prop_lon_rad = np.deg2rad(df_valid[lon_col].to_numpy())
        prop_X = np.vstack([prop_lat_rad, prop_lon_rad]).T
        finite_mask = np.isfinite(prop_lat_rad) & np.isfinite(prop_lon_rad)

        if finite_mask.any():
            prop_X_f = prop_X[finite_mask]
            idx_valid_f = df_valid.index.to_numpy()[finite_mask]  # MultiIndexラベル
            years_valid = df_valid[year_col].to_numpy(dtype=np.int64)[finite_mask]

            R = 6_371_000.0

            # --- 4) yearごとに処理（yearごとに land_gdf を 0除外してNN） ---
            unique_years = np.unique(years_valid)

            for y in unique_years:
                # 対象行（このyearの物件）
                rows_mask = (years_valid == y)
                if not rows_mask.any():
                    continue

                col_y = f'land_price_{int(y)}'
                col_y_1 = f'land_price_{int(y) - 1}'

                if col_y not in land_gdf.columns or col_y_1 not in land_gdf.columns:
                    # 列がなければこのyearは作れないのでスキップ
                    continue

                # --- 4-1) 標準地を「当年>0 かつ 前年>0」に限定 ---
                land_price_y = land_gdf[col_y].to_numpy(dtype=float)
                land_price_y1 = land_gdf[col_y_1].to_numpy(dtype=float)

                land_ok = (land_price_y > 0) & (land_price_y1 > 0)
                if land_ok.sum() < n_neighbors:
                    # 近傍が作れない（点が少ない）→ このyearの行は NaN のまま
                    continue

                land_sub = land_gdf.loc[land_ok]

                # 座標
                land_lon_rad = np.deg2rad(land_sub.geometry.x.to_numpy())
                land_lat_rad = np.deg2rad(land_sub.geometry.y.to_numpy())
                land_X = np.vstack([land_lat_rad, land_lon_rad]).T

                # 値（subsetに対応）
                land_price_y_sub = land_sub[col_y].to_numpy(dtype=float)
                land_price_y1_sub = land_sub[col_y_1].to_numpy(dtype=float)

                # --- 4-2) NN構築（yearごと） ---
                nn = NearestNeighbors(n_neighbors=n_neighbors, metric='haversine')
                nn.fit(land_X)

                # --- 4-3) 物件→近傍検索（このyearの行だけ） ---
                Xq = prop_X_f[rows_mask]
                idx_q_labels = idx_valid_f[rows_mask]

                distances_rad, indices = nn.kneighbors(Xq)  # (m, k)
                distances_m = distances_rad * R

                # land_price を (m,k) へ
                prices = land_price_y_sub[indices]
                prices_prev = land_price_y1_sub[indices]

                # 最近傍（1点目）
                nearest_prices = prices[:, 0]
                nearest_prev_prices = prices_prev[:, 0]
                nearest_dists = distances_m[:, 0]

                # 距離加重平均（ゼロ距離対応）
                d_m = distances_m
                zero_mask = (d_m == 0)
                has_zero = zero_mask.any(axis=1)

                w = np.zeros_like(d_m, dtype=float)
                nz = ~zero_mask
                w[nz] = 1.0 / d_m[nz]

                w_sum = w.sum(axis=1, keepdims=True)
                w_sum = np.where(w_sum == 0, 1.0, w_sum)
                w = w / w_sum

                wp = (w * prices).sum(axis=1)
                wp_prev = (w * prices_prev).sum(axis=1)

                if has_zero.any():
                    rows = np.where(has_zero)[0]
                    for r in rows:
                        z = zero_mask[r]
                        wp[r] = prices[r, z].mean()
                        wp_prev[r] = prices_prev[r, z].mean()

                # --- 4-4) 代入（MultiIndexラベルで安全に） ---
                nearest_all.loc[idx_q_labels] = nearest_prices
                nearest_prev_all.loc[idx_q_labels] = nearest_prev_prices
                dist_all.loc[idx_q_labels] = nearest_dists
                weighted_all.loc[idx_q_labels] = wp
                weighted_prev_all.loc[idx_q_labels] = wp_prev

    # --- 5) 列付与 ---
    out = combined.copy()
    out['nearest_land_price'] = nearest_all
    out['distance_to_landpoint_m'] = dist_all
    out['log_land_price'] = np.log1p(out['nearest_land_price'])

    out['weighted_land_price_3'] = weighted_all
    out['log_weighted_land_price_3'] = np.log1p(out['weighted_land_price_3'])

    out['nearest_land_price_prev'] = nearest_prev_all
    out['weighted_land_price_3_prev'] = weighted_prev_all

    # yoy/dlog（前年がNaNの行は自然にNaNになる）
    eps = 1.0
    denom_n = np.maximum(out['nearest_land_price_prev'].to_numpy(), eps)
    denom_w = np.maximum(out['weighted_land_price_3_prev'].to_numpy(), eps)

    out['land_price_yoy_nearest'] = (out['nearest_land_price'] - out['nearest_land_price_prev']) / denom_n
    out['land_price_yoy_w3'] = (out['weighted_land_price_3'] - out['weighted_land_price_3_prev']) / denom_w
    out['land_price_dlog_nearest'] = np.log1p(out['nearest_land_price']) - np.log1p(out['nearest_land_price_prev'])
    out['land_price_dlog_w3'] = np.log1p(out['weighted_land_price_3']) - np.log1p(out['weighted_land_price_3_prev'])

    # --- 6) train/test に戻す（元index保持） ---
    train_out = out.xs('train', level='__set__')
    test_out = out.xs('test', level='__set__')

    assert train_out.index.equals(train_df.index)
    assert test_out.index.equals(test_df.index)

    return train_out, test_out


In [11]:
train_df_geo, test_df_geo = add_land_price_features_haversine(
    train_df=train_df_geo,
    test_df=test_df_geo,
    land_gdf=land_gdf,
    lat_col='lat_jgd',
    lon_col='lon_jgd',
    year_col='target_year'
)

In [12]:
del land_gdf
gc.collect()

44

## 1kmメッシュ将来人口

In [13]:
fu_pop_path = gis_path + '人口メッシュ/1km_mesh_2024_GEOJSON/future_pop_1km.parquet'

In [14]:
fu_pop_df = gpd.read_parquet(fu_pop_path)

In [15]:
# #NOTE: 歪みが気になるのであればclipを用いる
# display(fu_pop_df[['pop_trend_slope', 'pop_trend_rate']].describe())
# q_low, q_high = fu_pop_df['pop_trend_rate'].quantile([0.01, 0.99])

# fu_pop_df['pop_trend_rate_clip'] = (
#     fu_pop_df['pop_trend_rate']
#     .clip(lower=q_low, upper=q_high)
# )

# display(fu_pop_df[['pop_trend_slope', 'pop_trend_rate_clip']].describe())

In [16]:
def _ensure_point_gdf_from_lonlat(df: pd.DataFrame,
                                 lon_col: str,
                                 lat_col: str,
                                 crs: str) -> gpd.GeoDataFrame:
    gdf = gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
        crs=crs
    )
    return gdf

def add_pop_knn_features(train_df_geo: pd.DataFrame,
                         test_df_geo: pd.DataFrame,
                         fu_pop_df: gpd.GeoDataFrame,
                         pop_cols: list[str],
                         lon_col: str = 'lon_jgd',
                         lat_col: str = 'lat_jgd',
                         in_crs: str = 'EPSG:4612',
                         work_crs: str = 'EPSG:6677',
                         ) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    train/test の各点に対して、人口メッシュ点の
    - 最近傍（nn）
    - k近傍の距離加重平均（idw）
    を付与する。
    """

    # --- 1) train/test を Point GeoDataFrame 化（EPSG:4612）
    tr_gdf = _ensure_point_gdf_from_lonlat(train_df_geo, lon_col, lat_col, in_crs)
    te_gdf = _ensure_point_gdf_from_lonlat(test_df_geo, lon_col, lat_col, in_crs)

    # --- 2) population を Point 化（Polygon の場合は centroid）
    pop_gdf = fu_pop_df.copy()
    if not all(pop_gdf.geometry.geom_type == 'Point'):
        pop_gdf = pop_gdf.to_crs(work_crs)
        pop_gdf['geometry'] = pop_gdf.geometry.centroid
        pop_gdf = pop_gdf.to_crs(in_crs)

    # --- 3) 作業用 CRS（メートル系）へ
    tr_m = tr_gdf.to_crs(work_crs)
    te_m = te_gdf.to_crs(work_crs)
    pop_m = pop_gdf.to_crs(work_crs)

    # --- 4) KNN 検索（メートル座標で）
    pop_xy = np.column_stack([pop_m.geometry.x.values, pop_m.geometry.y.values])
    nn = NearestNeighbors(n_neighbors=3, algorithm='auto', metric='euclidean')
    nn.fit(pop_xy)

    def _apply(gdf_m: gpd.GeoDataFrame, prefix: str) -> pd.DataFrame:
        q_xy = np.column_stack([gdf_m.geometry.x.values, gdf_m.geometry.y.values])
        dists, idxs = nn.kneighbors(q_xy, return_distance=True)  # shapes: (n, k)

        out = pd.DataFrame(index=gdf_m.index)

        # 最近傍距離
        out[f'{prefix}_dist_nn_m'] = dists[:, 0].astype(np.float32)

        # 各 pop_col について nn を作る
        for col in pop_cols:
            vals = pop_m[col].values[idxs]  # (n, k)
            out[f'{col}_nn'] = vals[:, 0]

        return out

    tr_feat = _apply(tr_m, 'pop')
    te_feat = _apply(te_m, 'pop')

    train_out = train_df_geo.join(tr_feat)
    test_out = test_df_geo.join(te_feat)

    return train_out, test_out


In [17]:
pop_cols = [
    'PTN_2020', # 2020年の総数人口
    'RTA_2025', # 2025年男女計0～14歳人口/比率
    'RTB_2025', # 2025年男女計15～64歳人口/比率
    'RTC_2025', # 2025年男女計65歳以上人口/比率
    'RTD_2025', # 2025年男女計75歳以上人口/比率
    'RTE_2025', # 2025年男女計80歳以上人口/比率
    'pop_trend_rate'
]

train_df_geo, test_df_geo = add_pop_knn_features(
    train_df_geo=train_df_geo,
    test_df_geo=test_df_geo,
    fu_pop_df=fu_pop_df,
    pop_cols=pop_cols
)

In [18]:
del fu_pop_df
gc.collect()

0

## 道路密度・道路延長メッシュ

In [19]:
road_mesh_path = gis_path + '道路密度・道路延長メッシュ/road_mesh_fe.parquet'
road_path = gis_path + '道路/N01-07L-48-01.0a_GML/N01-07L-2K_Road.shp'

In [20]:
road_mesh_df = gpd.read_parquet(road_mesh_path)
road_df = read_dataframe(road_path)

road_df = road_df.set_crs('EPSG:4612')

In [21]:
from __future__ import annotations

from dataclasses import dataclass
from typing import Iterable, Sequence

@dataclass(frozen=True)
class RoadLineFeatureConfig:
    # 距離・バッファ計算用の投影座標系（メートル単位）
    proj_epsg: int = 6677

    # road_df の道路種別コード列
    road_type_col: str = 'N01_001'

    # 主要道路とみなすコード（要件次第で調整）
    # 例: 1=高速, 2=一般国道, 3=主要地方道, 5=特例都道 などを含める
    major_codes: tuple[int, ...] = (1, 2, 3, 5)

    # 近傍カウント/延長を作る半径（m）
    radii_m: tuple[int, ...] = (100, 300, 500)

    # 半径内の延長（m）を計算するか（重い）
    compute_length_in_buffer: bool = False

    # sjoin の集計で「同一路線名をユニークカウント」したい場合に使う列（なければ None）
    # 例: 'N01_002'（路線名）など
    unique_key_col: str | None = None


def _to_points_gdf(
    df: pd.DataFrame,
    lon_col: str,
    lat_col: str,
    crs: str = 'EPSG:4612',
) -> gpd.GeoDataFrame:
    gdf = gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
        crs=crs,
    )
    return gdf


def _safe_int_series(s: pd.Series) -> pd.Series:
    # 'unknown' 等が混ざっても落ちないようにする
    return (
        s.replace('unknown', pd.NA)
         .astype('Int64')
    )


def _nearest_distance(
    pts_p: gpd.GeoDataFrame,
    lines_p: gpd.GeoDataFrame,
    out_col: str,
) -> pd.Series:
    if lines_p is None or len(lines_p) == 0:
        return pd.Series(np.nan, index=pts_p.index, name=out_col)

    joined = gpd.sjoin_nearest(
        pts_p[['geometry']],
        lines_p[['geometry']],
        how='left',
        distance_col=out_col,
    )

    # 同距離などで点indexが重複するケースがあるので、点ごとに最小距離に潰す
    s = joined[out_col]
    if not s.index.is_unique:
        s = s.groupby(level=0).min()

    # pts_p の index に整列して返す（欠けは NaN）
    s = s.reindex(pts_p.index)

    return s.astype('float32')

def _count_lines_within_radius(
    pts_p: gpd.GeoDataFrame,
    lines_p: gpd.GeoDataFrame,
    radius_m: int,
    out_col: str,
    unique_key_col: str | None = None,
) -> pd.Series:
    """
    半径バッファ内に intersect する road セグメント数（または unique_key_col のユニーク数）
    """
    if lines_p is None or len(lines_p) == 0:
        return pd.Series(0, index=pts_p.index, name=out_col, dtype='int32')

    buf = pts_p[['geometry']].copy()
    buf['geometry'] = buf.geometry.buffer(radius_m)

    joined = gpd.sjoin(
        buf,
        lines_p,
        how='left',
        predicate='intersects',
    )

    if unique_key_col is not None and unique_key_col in joined.columns:
        cnt = joined.groupby(level=0)[unique_key_col].nunique(dropna=True)
    else:
        # セグメント（行）数。index_right のユニークで数えるのが安定
        if 'index_right' in joined.columns:
            cnt = joined.groupby(level=0)['index_right'].nunique(dropna=True)
        else:
            cnt = joined.groupby(level=0).size()

    out = pd.Series(0, index=pts_p.index, name=out_col, dtype='int32')
    out.loc[cnt.index] = cnt.astype('int32')
    return out


def _length_lines_within_radius(
    pts_p: gpd.GeoDataFrame,
    lines_p: gpd.GeoDataFrame,
    radius_m: int,
    out_col: str,
) -> pd.Series:
    """
    半径バッファ内の道路延長（m）合計。
    注意: 計算コストが高いので radii が大きい・件数が多い場合は重くなる。
    """
    if lines_p is None or len(lines_p) == 0:
        return pd.Series(0.0, index=pts_p.index, name=out_col, dtype='float32')

    buf = pts_p[['geometry']].copy()
    buf['geometry'] = buf.geometry.buffer(radius_m)

    joined = gpd.sjoin(
        buf,
        lines_p[['geometry']],
        how='left',
        predicate='intersects',
    )

    # index の対応でバッファ形状を引けるようにする
    buf_geom = buf.geometry

    # intersect した道路をバッファでクリップして長さ
    # （ベクトル化しにくいので apply。重い）
    def _clip_len(row) -> float:
        if pd.isna(row.get('index_right')):
            return 0.0
        g_line = row['geometry_right']
        g_buf = buf_geom.loc[row.name]
        return float(g_line.intersection(g_buf).length)

    # sjoin の結果に right geometry が入らない場合があるので付与
    joined = joined.rename(columns={'geometry': 'geometry_left'})
    joined = joined.merge(
        lines_p[['geometry']].rename(columns={'geometry': 'geometry_right'}),
        left_on='index_right',
        right_index=True,
        how='left',
    )

    lens = joined.apply(_clip_len, axis=1)
    agg = lens.groupby(level=0).sum()

    out = pd.Series(0.0, index=pts_p.index, name=out_col, dtype='float32')
    out.loc[agg.index] = agg.astype('float32')
    return out


def add_road_features(
    df: pd.DataFrame,
    road_poly: gpd.GeoDataFrame,
    road_line: gpd.GeoDataFrame,
    lon_col: str = 'lon_jgd',
    lat_col: str = 'lat_jgd',
    poly_join_predicate: str = 'within',
    poly_keep_cols: Sequence[str] | None = None,
    line_cfg: RoadLineFeatureConfig = RoadLineFeatureConfig(),
) -> pd.DataFrame:
    """
    df（train_df_geo / test_df_geo想定）に対して、
      A) road_mesh_df(POLYGON) 由来の列を sjoin で付与
      B) road_df(LINESTRING) 由来の近傍本数・主要道路距離等を付与
    をまとめて実行する。

    返り値: 元の df と同じ index の DataFrame（geometry は持たない）
    """
    out = df.copy()

    # ----------------------------
    # A) POLYGON（メッシュ特徴量）を sjoin
    # ----------------------------
    pts_4612 = _to_points_gdf(out, lon_col=lon_col, lat_col=lat_col, crs='EPSG:4612')
    poly_4612 = road_poly.to_crs('EPSG:4612')

    joined_poly = gpd.sjoin(
        pts_4612[['geometry']],
        poly_4612,
        how='left',
        predicate=poly_join_predicate,
    )

    # 必要な列だけ付与（index_right 等の管理列は落とす）
    if poly_keep_cols is None:
        # 典型的にはあなたの road_mesh_df が保持している特徴量列をすべて付与でOK
        drop_cols = [c for c in ['index_right', 'geometry'] if c in joined_poly.columns]
        poly_cols = [c for c in joined_poly.columns if c not in drop_cols]
    else:
        poly_cols = [c for c in poly_keep_cols if c in joined_poly.columns]

    out = out.join(joined_poly[poly_cols])

    # ----------------------------
    # B) LINESTRING（近傍・距離）特徴量
    # ----------------------------
    # 投影CRSへ
    pts_p = pts_4612.to_crs(epsg=line_cfg.proj_epsg)

    road_line_4612 = road_line
    if road_line_4612.crs is None:
        # ここはあなたのデータ取得仕様に合わせて変更可
        road_line_4612 = road_line_4612.set_crs('EPSG:4612')

    lines_p = road_line_4612.to_crs(epsg=line_cfg.proj_epsg)

    # 道路種別コードを整形
    if line_cfg.road_type_col in lines_p.columns:
        lines_p = lines_p.copy()
        lines_p[line_cfg.road_type_col] = _safe_int_series(lines_p[line_cfg.road_type_col])

    # サブセット
    major_mask = pd.Series(False, index=lines_p.index)
    if line_cfg.road_type_col in lines_p.columns:
        major_mask = lines_p[line_cfg.road_type_col].isin(list(line_cfg.major_codes))

    lines_major = lines_p.loc[major_mask].copy()
    lines_highway = lines_p.loc[lines_p[line_cfg.road_type_col].isin([1])] if line_cfg.road_type_col in lines_p.columns else lines_p.iloc[0:0]

    # 最近傍距離
    out['dist_to_road_any_m'] = _nearest_distance(pts_p, lines_p, 'dist_to_road_any_m')
    out['dist_to_road_major_m'] = _nearest_distance(pts_p, lines_major, 'dist_to_road_major_m')
    out['dist_to_road_highway_m'] = _nearest_distance(pts_p, lines_highway, 'dist_to_road_highway_m')

    # 半径内の道路本数（セグメント数 or 路線名ユニーク数）
    for r in line_cfg.radii_m:
        out[f'road_cnt_any_in_{r}m'] = _count_lines_within_radius(
            pts_p,
            lines_p if line_cfg.unique_key_col is None else lines_p[[line_cfg.unique_key_col, 'geometry']].copy(),
            radius_m=r,
            out_col=f'road_cnt_any_in_{r}m',
            unique_key_col=line_cfg.unique_key_col,
        )
        out[f'road_cnt_major_in_{r}m'] = _count_lines_within_radius(
            pts_p,
            lines_major if line_cfg.unique_key_col is None else lines_major[[line_cfg.unique_key_col, 'geometry']].copy(),
            radius_m=r,
            out_col=f'road_cnt_major_in_{r}m',
            unique_key_col=line_cfg.unique_key_col,
        )

        # （任意）半径内延長
        if line_cfg.compute_length_in_buffer:
            out[f'road_len_any_in_{r}m'] = _length_lines_within_radius(
                pts_p,
                lines_p,
                radius_m=r,
                out_col=f'road_len_any_in_{r}m',
            )
            out[f'road_len_major_in_{r}m'] = _length_lines_within_radius(
                pts_p,
                lines_major,
                radius_m=r,
                out_col=f'road_len_major_in_{r}m',
            )

    return out


In [22]:
cfg = RoadLineFeatureConfig(
    proj_epsg=6677,
    major_codes=(1, 2, 3, 5),
    radii_m=(100, 300, 500),
    compute_length_in_buffer=False,     # 重いのでまずはOFF推奨
    unique_key_col=None,               # 'N01_002' で路線名ユニーク数にしたいなら 'N01_002'
)

train_df_geo = add_road_features(
    df=train_df_geo,
    road_poly=road_mesh_df,
    road_line=road_df,
    lon_col='lon_jgd',
    lat_col='lat_jgd',
    poly_keep_cols=None,   # 付けたいメッシュ列だけに絞るなら ['road_len_total', ...] を指定
    line_cfg=cfg,
)

test_df_geo = add_road_features(
    df=test_df_geo,
    road_poly=road_mesh_df,
    road_line=road_df,
    lon_col='lon_jgd',
    lat_col='lat_jgd',
    poly_keep_cols=None,
    line_cfg=cfg,
)


In [23]:
del cfg, road_mesh_df, road_df
gc.collect()

0

## 用途地域

In [24]:
youto_path = gis_path + '用途地域/youto.parquet'

In [25]:
youto_df = gpd.read_parquet(youto_path)

In [26]:
def add_youto_features_sjoin(
    df: pd.DataFrame,
    youto_poly: gpd.GeoDataFrame,
    *,
    lon_col: str = 'lon_jgd',
    lat_col: str = 'lat_jgd',
    df_crs: str = 'EPSG:4612',   # ★ 明示的に 4612
    keep_cols: tuple[str, ...] = (
        'A29_004', 'A29_005', 'A29_006', 'A29_007',
        'zone_group', 'zone_residential_rank',
        'is_lowrise_residential',
        'kenpei', 'youseki', 'kenpei_missing', 'youseki_missing',
        'kenpei_bin', 'youseki_bin', 'density_cat'
    ),
    predicate_primary: str = 'intersects',
    predicate_fallback: str | None = None,
) -> pd.DataFrame:

    out = df.copy()

    # --- point gdf: pt_id を作って index 依存を消す ---
    gdf_pts = gpd.GeoDataFrame(
    out[[lon_col, lat_col]].copy(),
    geometry=gpd.points_from_xy(out[lon_col], out[lat_col]),
    crs=df_crs,   # ★ 4612
    )
    gdf_pts['pt_id'] = np.arange(len(gdf_pts), dtype=np.int64)


    # --- poly ---
    youto = youto_poly.copy()
    if youto.crs is None:
        raise ValueError('youto_poly.crs が None です。CRS を設定してください。')

    if youto.crs.to_string() != df_crs:
        youto = youto.to_crs(df_crs)

    cols_exist = [c for c in keep_cols if c in youto.columns]
    youto_small = youto[cols_exist + ['geometry']].copy()

    # 代表選択キー（厳しい規制を優先：小さいほど優先）
    if 'kenpei' not in youto_small.columns and 'A29_006' in youto_small.columns:
        youto_small['kenpei'] = pd.to_numeric(youto_small['A29_006'], errors='coerce').astype('float64')
    if 'youseki' not in youto_small.columns and 'A29_007' in youto_small.columns:
        youto_small['youseki'] = pd.to_numeric(youto_small['A29_007'], errors='coerce').astype('float64')

    youto_small['kenpei_key'] = youto_small.get('kenpei', pd.Series(np.nan, index=youto_small.index)).fillna(1e9)
    youto_small['youseki_key'] = youto_small.get('youseki', pd.Series(np.nan, index=youto_small.index)).fillna(1e9)

    # --- primary join ---
    j1 = gpd.sjoin(
        gdf_pts[['pt_id', 'geometry']],
        youto_small,
        how='left',
        predicate=predicate_primary,
    )

    # --- fallback join（必要なときだけ）---
    if predicate_fallback is not None:
        miss = j1['index_right'].isna() if 'index_right' in j1.columns else j1.isna().any(axis=1)
        miss_pt_ids = j1.loc[miss, 'pt_id'].unique()

        if len(miss_pt_ids) > 0:
            pts_miss = gdf_pts.loc[gdf_pts['pt_id'].isin(miss_pt_ids), ['pt_id', 'geometry']]
            j2 = gpd.sjoin(
                pts_miss,
                youto_small,
                how='left',
                predicate=predicate_fallback,
            )
            j_all = pd.concat([j1, j2], axis=0, ignore_index=True)
        else:
            j_all = j1
    else:
        j_all = j1

    # --- 複数マッチを 1件に落とす（pt_id単位）---
    j_all = j_all.sort_values(['pt_id', 'kenpei_key', 'youseki_key'])
    j_best = j_all.groupby('pt_id', sort=False).head(1).copy()

    # --- df の行順に合わせて join ---
    j_best = j_best.set_index('pt_id')

    drop_cols = [c for c in ['index_right', 'geometry', 'kenpei_key', 'youseki_key'] if c in j_best.columns]
    j_best = j_best.drop(columns=drop_cols, errors='ignore')

    out2 = out.reset_index(drop=True).join(j_best, how='left')
    out2.index = out.index

    return out2


In [27]:
train_df_geo = add_youto_features_sjoin(train_df_geo, youto_df)
test_df_geo  = add_youto_features_sjoin(test_df_geo,  youto_df)

print(train_df_geo['A29_005'].isna().mean())
print(test_df_geo['A29_005'].isna().mean())
print(train_df_geo['density_cat'].isna().mean())
print(test_df_geo['density_cat'].isna().mean())

0.1523642298941537
0.14870549730071062
0.1523642298941537
0.14870549730071062


In [28]:
del youto_df
gc.collect()

0

## 都市計画

In [29]:
koudoti_path = gis_path + '都市計画/koudoti.parquet'
chikukei_path = gis_path + '都市計画/chikukei.parquet'
tochiku_path = gis_path + '都市計画/tochiku.parquet'
koudori_path = gis_path + '都市計画/koudori.parquet'
toshisaisei_path = gis_path + '都市計画/toshisaisei.parquet'
tokureiyouseki_path = gis_path + '都市計画/tokureiyouseki.parquet'
kousoujyukyo_path = gis_path + '都市計画/kousoujyukyo.parquet'
tokuteibousai_path = gis_path + '都市計画/tokuteibousai.parquet'
fukkousaiseikyoten_path = gis_path + '都市計画/fukkousaiseikyoten.parquet'
senbiki_path = gis_path + '都市計画/senbiki_df.parquet'
bouka_path = gis_path + '都市計画/bouka_df.parquet'

In [30]:
koudoti_df = gpd.read_parquet(koudoti_path)
chikukei_df = gpd.read_parquet(chikukei_path)
tochiku_df = gpd.read_parquet(tochiku_path)
koudori_df = gpd.read_parquet(koudori_path)
toshisaisei_df = gpd.read_parquet(toshisaisei_path)
tokureiyouseki_df = gpd.read_parquet(tokureiyouseki_path)
kousoujyukyo_df = gpd.read_parquet(kousoujyukyo_path)
tokuteibousai_df = gpd.read_parquet(tokuteibousai_path)
fukkousaiseikyoten_df = gpd.read_parquet(fukkousaiseikyoten_path)
senbiki_df = gpd.read_parquet(senbiki_path)
bouka_df = gpd.read_parquet(bouka_path)

In [31]:
def _make_points(df: pd.DataFrame, lon_col: str, lat_col: str, crs: str) -> gpd.GeoDataFrame:
    pts = gpd.GeoDataFrame(
        df[[lon_col, lat_col]].copy(),
        geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
        crs=crs,
    )
    pts['__pt_id__'] = np.arange(len(pts), dtype=np.int64)
    return pts


def _sjoin_flags_one_layer(
    df: pd.DataFrame,
    poly: gpd.GeoDataFrame,
    flag_cols: list[str],
    *,
    lon_col: str = 'lon_jgd',
    lat_col: str = 'lat_jgd',
    df_crs: str = 'EPSG:4612',
    predicate_main: str = 'within',
    predicate_fallback: str = 'intersects',
) -> pd.DataFrame:
    """
    poly(フラグ列+geometry) を df(point) に空間結合し、flag_cols を付与。
    複数マッチは OR(max) で集約。
    """
    if poly is None or len(poly) == 0:
        out = df.copy()
        for c in flag_cols:
            if c not in out.columns:
                out[c] = 0
            out[c] = out[c].fillna(0).astype('int8')
        return out

    if poly.crs is None:
        raise ValueError('poly.crs is None. set_crs / to_crs を確認してください。')

    poly_ = poly.to_crs(df_crs).copy()
    pts = _make_points(df, lon_col, lat_col, df_crs)

    # main join
    j1 = gpd.sjoin(
        pts[['__pt_id__', 'geometry']],
        poly_[flag_cols + ['geometry']],
        how='left',
        predicate=predicate_main,
    )

    # fallback join（mainで当たらなかったptだけ）
    if predicate_fallback is not None:
        miss_ids = j1.loc[j1['index_right'].isna(), '__pt_id__'].unique()
        if len(miss_ids) > 0:
            pts_miss = pts.loc[pts['__pt_id__'].isin(miss_ids), ['__pt_id__', 'geometry']]
            j2 = gpd.sjoin(
                pts_miss,
                poly_[flag_cols + ['geometry']],
                how='left',
                predicate=predicate_fallback,
            )
            j = pd.concat([j1, j2], ignore_index=True)
        else:
            j = j1
    else:
        j = j1

    # OR集約（フラグなのでmaxでよい）
    for c in flag_cols:
        j[c] = pd.to_numeric(j[c], errors='coerce')

    agg = (
        j.groupby('__pt_id__', sort=False)[flag_cols]
        .max()
        .reset_index()
    )

    out = df.reset_index(drop=True).copy()
    out = out.join(agg.set_index('__pt_id__'), how='left')
    out.index = df.index

    for c in flag_cols:
        out[c] = out[c].fillna(0).astype('int8')

    return out


In [32]:
def add_cityplan_layers(
    df: pd.DataFrame,
    layers: dict[str, tuple[gpd.GeoDataFrame, list[str]]],
    *,
    lon_col: str = 'lon_jgd',
    lat_col: str = 'lat_jgd',
    df_crs: str = 'EPSG:4612',
    predicate_main: str = 'within',
    predicate_fallback: str = 'intersects',
) -> pd.DataFrame:
    out = df.copy()

    for name, (poly, cols) in layers.items():
        # 念のため cols が poly にあるかチェック
        if poly is not None and len(poly) > 0:
            missing = [c for c in cols if c not in poly.columns]
            if missing:
                raise KeyError(f'layer={name} missing cols={missing}')

        out = _sjoin_flags_one_layer(
            out,
            poly,
            cols,
            lon_col=lon_col,
            lat_col=lat_col,
            df_crs=df_crs,
            predicate_main=predicate_main,
            predicate_fallback=predicate_fallback,
        )

    return out


In [33]:
layers = {
    'senbiki': (senbiki_df, ['is_urbanized_area', 'is_urban_control_area']),
    'bouka': (bouka_df, ['is_fireproof_area', 'is_quasi_fireproof_area']),
    'koudoti': (koudoti_df, ['has_height_limit']),
    'chikukei': (chikukei_df, ['has_district_plan']),
    'tochiku': (tochiku_df, ['is_land_readjustment_area']),
    'koudori': (koudori_df, ['is_high_utilization_area']),
    'toshisaisei': (toshisaisei_df, ['is_urban_renaissance_area']),
    'tokureiyouseki': (tokureiyouseki_df, ['is_special_far_area']),
    'kousoujyukyo': (kousoujyukyo_df, ['is_highrise_residential_area']),
    'tokuteibousai': (tokuteibousai_df, ['is_disaster_prevention_block']),
    'fukkousaiseikyoten': (fukkousaiseikyoten_df, ['is_redevelopment_core_area']),
}

train_df_geo = add_cityplan_layers(train_df_geo, layers, lon_col='lon_jgd', lat_col='lat_jgd')
test_df_geo  = add_cityplan_layers(test_df_geo,  layers, lon_col='lon_jgd', lat_col='lat_jgd')

In [34]:
del layers
gc.collect()

0

## 標高

In [35]:
height_path = gis_path + '標高/height.parquet'

In [36]:
height_df = gpd.read_parquet(height_path)

In [37]:
def add_height_features(
    df: pd.DataFrame,
    height_poly: gpd.GeoDataFrame,
    lon_col='lon_jgd',
    lat_col='lat_jgd',
):
    gdf = gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df[lon_col], df[lat_col]),
        crs='EPSG:4612',
    )

    height_poly = height_poly.set_crs('EPSG:4612')

    joined = gpd.sjoin(
        gdf,
        height_poly,
        how='left',
        predicate='within',
    )

    return pd.DataFrame(joined.drop(columns='geometry'))


In [38]:
HEIGHT_NUM_COLS = [
    'G04c_002', 'G04c_003', 'G04c_004',
    'G04c_006', 'G04c_008', 'G04c_010'
]

def coerce_height_numeric(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    for c in HEIGHT_NUM_COLS:
        if c in out.columns:
            out[c] = pd.to_numeric(
                out[c].replace('unknown', np.nan),
                errors='coerce'
            )
    return out

In [39]:
# 1) 結合（sjoin）
train_df_geo = add_height_features(train_df_geo, height_df)
test_df_geo  = add_height_features(test_df_geo, height_df)

# 2) 数値化（unknown -> NaN, object -> float）
train_df_geo = coerce_height_numeric(train_df_geo)
test_df_geo  = coerce_height_numeric(test_df_geo)

# 3) 特徴量作成
for df in [train_df_geo, test_df_geo]:
    df['elev_mean']   = df['G04c_002']
    df['elev_range']  = df['G04c_003'] - df['G04c_004']

    df['slope_mean']  = df['G04c_010']
    df['slope_max']   = df['G04c_006']
    df['slope_range'] = df['G04c_006'] - df['G04c_008']


In [40]:
del height_df
gc.collect()

0

## 駅別乗降者数

In [41]:
file = Path(gis_path + '駅別乗降者数/S12-24_GML/S12-24_NumberOfPassengers.geojson')
eki_gdf = read_dataframe(file) # 高速化しないとめちゃ時間かかる
print(eki_gdf.crs)

EPSG:6668


In [42]:
eki_gdf = eki_gdf.rename(columns={
    'S12_001': '駅名',
    'S12_003': '路線名',
    'S12_037': '乗降客数_2018',
    'S12_041': '乗降客数_2019',
    'S12_045': '乗降客数_2020',
    'S12_049': '乗降客数_2021',
    'S12_053': '乗降客数_2022',
    'S12_057': '乗降客数_2023',
})

In [43]:
year_cols = [
    '乗降客数_2018', '乗降客数_2019',
    '乗降客数_2020', '乗降客数_2021',
    '乗降客数_2022', '乗降客数_2023'
]

eki_gdf[year_cols] = eki_gdf[year_cols].replace(0, np.nan)

normal_cols = [
    '乗降客数_2018',
    '乗降客数_2019',
    '乗降客数_2022',
    '乗降客数_2023',
]

eki_gdf['passenger_normal'] = (
    eki_gdf[normal_cols]
    .median(axis=1, skipna=True)
)

In [44]:
station_agg = (
    eki_gdf
    .dropna(subset=['passenger_normal'])
    .groupby('駅名', as_index=False)
    .agg(
        passenger_normal=('passenger_normal', 'sum'),
        geometry=('geometry', 'first')  # ほぼ同一なので代表
    )
)

station_agg['passenger_normal_log'] = np.log1p(
    station_agg['passenger_normal']
)

station_agg = gpd.GeoDataFrame(
    station_agg,
    geometry='geometry',
    crs='EPSG:6668'   # 駅データのCRS
)

station_agg['geometry'] = station_agg.geometry.centroid

station_agg['lon'] = station_agg.geometry.x
station_agg['lat'] = station_agg.geometry.y

In [45]:
from sklearn.neighbors import BallTree

def add_station_power_summax3(
    df: pd.DataFrame,
    station_agg: pd.DataFrame,
    lon_col: str = 'lon_jgd',
    lat_col: str = 'lat_jgd',
    station_lon_col: str = 'lon',
    station_lat_col: str = 'lat',
    passenger_col: str = 'passenger_normal_log',
    k: int = 3,
    tau_km: float = 1.0,
) -> pd.DataFrame:
    """
    物件ごとに近傍3駅の駅力スコアを作成（ソフト減衰）
      - station_power_sum3
      - station_power_max3

    score_i = passenger_log_i * exp(-distance_km / tau_km)

    前提:
      - df: 物件DF（lon_jgd / lat_jgd）
      - station_agg: 1駅=1行（lon / lat / passenger_normal_log）
    """
    out = df.copy()

    # --- 物件座標（ラジアン） ---
    prop_xy = np.deg2rad(
        out[[lat_col, lon_col]].to_numpy()
    )

    # --- 駅座標（ラジアン） ---
    station_xy = np.deg2rad(
        station_agg[[station_lat_col, station_lon_col]].to_numpy()
    )

    # --- BallTree (haversine) ---
    tree = BallTree(station_xy, metric='haversine')

    # k近傍探索（距離はラジアン）
    dist_rad, idx = tree.query(prop_xy, k=k)

    # km に変換
    dist_km = dist_rad * 6371.0

    # 駅の passenger_log
    pax = station_agg[passenger_col].to_numpy()

    # --- スコア計算 ---
    scores = np.zeros_like(dist_km)

    for i in range(k):
        scores[:, i] = pax[idx[:, i]] * np.exp(-dist_km[:, i] / tau_km)

    # --- 集約 ---
    out['station_power_sum3'] = scores.sum(axis=1)
    out['station_power_max3'] = scores.max(axis=1)

    return out


In [46]:
train_df_geo = add_station_power_summax3(
    train_df_geo,
    station_agg,
    lon_col='lon_jgd',
    lat_col='lat_jgd',
    tau_km=1.0,   # まずは 1km
)

test_df_geo = add_station_power_summax3(
    test_df_geo,
    station_agg,
    lon_col='lon_jgd',
    lat_col='lat_jgd',
    tau_km=1.0,
)

In [47]:
del station_agg
gc.collect()

0

## 災害危険区域

In [48]:
disaster_path = gis_path + '災害危険区域/disaster.parquet'

In [49]:
disaster_df = gpd.read_parquet(disaster_path)

In [50]:
def add_disaster_features_sjoin(
    df: pd.DataFrame,
    disaster_poly: gpd.GeoDataFrame,
    *,
    lon_col: str = 'lon_jgd',
    lat_col: str = 'lat_jgd',
    df_crs: str = 'EPSG:4612',
    reason_col: str = 'A48_007',
    predicate_primary: str = 'within',
    predicate_fallback: str | None = None,
    alpha_multi: float = 0.3,
    score_clip_max: float | None = 2.0,
    weight_map: dict[int, float] | None = None,
) -> pd.DataFrame:
    """
    災害危険区域（A48）ポリゴンと物件代表点を空間結合し、
    指定理由（A48_007）を重み付けして災害スコアを付与する。

    付与する列:
      - disaster_hit (0/1)
      - disaster_n_types
      - disaster_score_sum
      - disaster_score_max
      - disaster_score (max + alpha*(n_types-1), optional clip)
    """
    out = df.copy()

    # --- point gdf ---
    gdf_pts = gpd.GeoDataFrame(
        out[[lon_col, lat_col]].copy(),
        geometry=gpd.points_from_xy(out[lon_col], out[lat_col]),
        crs=df_crs,
    )
    gdf_pts['pt_id'] = np.arange(len(gdf_pts), dtype=np.int64)

    # --- poly ---
    dis = disaster_poly.copy()
    if dis.crs is None:
        raise ValueError('disaster_poly.crs が None です。CRS を設定してください。')

    if dis.crs.to_string() != df_crs:
        dis = dis.to_crs(df_crs)

    # reason_col を確実に持つ
    if reason_col not in dis.columns:
        raise ValueError(f'reason_col={reason_col} が disaster_poly に存在しません。')

    dis_small = dis[[reason_col, 'geometry']].copy()

    # --- join primary ---
    j1 = gpd.sjoin(
        gdf_pts[['pt_id', 'geometry']],
        dis_small,
        how='left',
        predicate=predicate_primary,
    )

    # --- fallback join（必要なら）---
    if predicate_fallback is not None:
        miss = j1['index_right'].isna()
        miss_pt_ids = j1.loc[miss, 'pt_id'].unique()

        if len(miss_pt_ids) > 0:
            pts_miss = gdf_pts.loc[gdf_pts['pt_id'].isin(miss_pt_ids), ['pt_id', 'geometry']]
            j2 = gpd.sjoin(
                pts_miss,
                dis_small,
                how='left',
                predicate=predicate_fallback,
            )
            j_all = pd.concat([j1, j2], axis=0, ignore_index=True)
        else:
            j_all = j1
    else:
        j_all = j1

    # --- 重み付け ---
    if weight_map is None:
        weight_map = {
            1: 0.8,  # 水害（河川）
            2: 0.7,  # 水害（海）
            3: 1.0,  # 水害（河川・海）
            4: 0.9,  # 急傾斜地崩壊等
            5: 0.9,  # 地すべり等
            6: 0.6,  # 火山災害
            7: 0.4,  # その他
        }

    j_all['reason_code'] = pd.to_numeric(j_all[reason_col], errors='coerce')
    j_all['reason_w'] = j_all['reason_code'].map(weight_map).astype('float64')

    # --- pt_id 単位に集計（複数マッチを“集計して残す”）---
    agg = (
        j_all.groupby('pt_id', as_index=False)
             .agg(
                 disaster_hit=('reason_code', lambda x: int(x.notna().any())),
                 disaster_n_types=('reason_code', lambda x: x.dropna().nunique()),
                 disaster_score_sum=('reason_w', 'sum'),
                 disaster_score_max=('reason_w', 'max'),
             )
    )

    # --- 元df順で戻す ---
    feat = pd.DataFrame({'pt_id': np.arange(len(out), dtype=np.int64)})
    feat = feat.merge(agg, on='pt_id', how='left')

    for c in ['disaster_hit', 'disaster_n_types', 'disaster_score_sum', 'disaster_score_max']:
        feat[c] = feat[c].fillna(0)

    # 複合リスク強調（max + alpha*(n_types-1)）
    feat['disaster_score'] = (
        feat['disaster_score_max'] + alpha_multi * np.maximum(feat['disaster_n_types'] - 1, 0)
    )

    if score_clip_max is not None:
        feat['disaster_score'] = feat['disaster_score'].clip(upper=score_clip_max)

    feat = feat.drop(columns=['pt_id'])

    out2 = out.reset_index(drop=True).join(feat, how='left')
    out2.index = out.index

    return out2


In [51]:
weight_map = {
    1: 0.8,  # 水害（河川）
    2: 0.7,  # 水害（海）
    3: 1.0,  # 水害（河川・海）
    4: 0.9,  # 急傾斜地崩壊等
    5: 0.9,  # 地すべり等
    6: 0.6,  # 火山災害
    7: 0.4,  # その他
}

train_df_geo = add_disaster_features_sjoin(train_df_geo, disaster_df, predicate_primary='within', weight_map=weight_map)
test_df_geo  = add_disaster_features_sjoin(test_df_geo,  disaster_df, predicate_primary='within', weight_map=weight_map)


In [52]:
rate_all = test_df_geo['disaster_hit'].mean()

print(f'train全体 disaster_hit率: {rate_all:.4%}')
print(f'該当件数: {test_df_geo["disaster_hit"].sum():,.0f} / {len(test_df_geo):,.0f}')


train全体 disaster_hit率: 0.5025%
該当件数: 565 / 112,437


In [53]:
del disaster_df
gc.collect()

0

## 学校

In [54]:
school_path = Path(gis_path + '学校/P29-23_GML/P29-23.geojson')
school_df = gpd.read_file(school_path)

In [55]:
school_use = school_df.loc[
    school_df['P29_007'] != 2
].copy()

school_use['school_type_norm'] = school_use['P29_003']

# 中等教育学校 → 中学校に寄せる
school_use.loc[
    school_use['school_type_norm'] == '16003',
    'school_type_norm'
] = '16002'

In [56]:
school_elem = school_use.loc[
    school_use['school_type_norm'] == '16001'
].copy()

school_junior = school_use.loc[
    school_use['school_type_norm'] == '16002'
].copy()

In [57]:
def add_school_features_from_gdfs(
    df: pd.DataFrame,
    school_elem: gpd.GeoDataFrame,
    school_junior: gpd.GeoDataFrame,
    *,
    lon_col: str = 'lon_jgd',
    lat_col: str = 'lat_jgd',
    df_crs: str = 'EPSG:4612',
    # 距離の計算用投影（m単位）
    proj_crs: str = 'EPSG:3857',
    # bin しきい値
    elem_threshold_m: int = 500,
    junior_threshold_m: int = 1000,
    # 欠損対策（通常は不要だが安全に）
    fill_missing_distance_m: float = 10_000.0,
    add_log_features: bool = True,
) -> pd.DataFrame:
    """
    物件代表点（lon/lat）に対し、school_elem（小学校）・school_junior（中学校）
    の最近傍距離を計算して特徴量追加する。

    追加列:
      - dist_elem_school_m
      - dist_junior_school_m
      - elem_school_500m
      - junior_school_1km
      - (optional) dist_elem_school_log, dist_junior_school_log
    """
    out = df.copy()

    # --- 入力チェック ---
    if lon_col not in out.columns or lat_col not in out.columns:
        raise ValueError(f'df に {lon_col}/{lat_col} がありません。')

    if school_elem is None or len(school_elem) == 0:
        raise ValueError('school_elem が空です。')
    if school_junior is None or len(school_junior) == 0:
        raise ValueError('school_junior が空です。')

    if school_elem.crs is None or school_junior.crs is None:
        raise ValueError('school_elem / school_junior の crs が None です。')

    # --- 物件点GDF ---
    gdf_pts = gpd.GeoDataFrame(
        out[[lon_col, lat_col]].copy(),
        geometry=gpd.points_from_xy(out[lon_col], out[lat_col]),
        crs=df_crs,
    )
    gdf_pts['pt_id'] = np.arange(len(gdf_pts), dtype=np.int64)

    # --- 距離計算は投影してm単位で ---
    pts_proj = gdf_pts.to_crs(proj_crs)
    elem_proj = school_elem.to_crs(proj_crs)
    junior_proj = school_junior.to_crs(proj_crs)

    # sjoin_nearest で最近傍距離（高速）
    j_elem = gpd.sjoin_nearest(
        pts_proj[['pt_id', 'geometry']],
        elem_proj[['geometry']],
        how='left',
        distance_col='dist_elem_school_m',
    )
    j_jun = gpd.sjoin_nearest(
        pts_proj[['pt_id', 'geometry']],
        junior_proj[['geometry']],
        how='left',
        distance_col='dist_junior_school_m',
    )

    # pt_id に揃えて距離列を抽出
    dist_elem = j_elem.set_index('pt_id')['dist_elem_school_m']
    dist_jun = j_jun.set_index('pt_id')['dist_junior_school_m']

    feat = pd.DataFrame({'pt_id': np.arange(len(out), dtype=np.int64)})
    feat = feat.join(dist_elem, on='pt_id')
    feat = feat.join(dist_jun, on='pt_id')

    # 欠損（通常は起きないが）
    feat['dist_elem_school_m'] = pd.to_numeric(feat['dist_elem_school_m'], errors='coerce').fillna(fill_missing_distance_m)
    feat['dist_junior_school_m'] = pd.to_numeric(feat['dist_junior_school_m'], errors='coerce').fillna(fill_missing_distance_m)

    # bin（閾値効果）
    feat['elem_school_500m'] = (feat['dist_elem_school_m'] <= elem_threshold_m).astype('int8')
    feat['junior_school_1km'] = (feat['dist_junior_school_m'] <= junior_threshold_m).astype('int8')

    # log（Tableau/非線形吸収用：入れるかは後で判断可）
    if add_log_features:
        feat['dist_elem_school_log'] = np.log1p(feat['dist_elem_school_m'])
        feat['dist_junior_school_log'] = np.log1p(feat['dist_junior_school_m'])

    feat = feat.drop(columns=['pt_id'])

    out2 = out.reset_index(drop=True).join(feat, how='left')
    out2.index = out.index
    return out2


In [58]:
train_df_geo = add_school_features_from_gdfs(train_df_geo, school_elem, school_junior)
test_df_geo  = add_school_features_from_gdfs(test_df_geo,  school_elem, school_junior)

In [59]:
del school_elem, school_junior
gc.collect()

0

## 病院

In [60]:
hospital_path = Path(gis_path + '病院/P04-20_GML/P04-20.geojson')
hospital_df = gpd.read_file(hospital_path)

In [61]:
def add_medical_features_from_gdf(
    df: pd.DataFrame,
    medical_gdf: gpd.GeoDataFrame,
    *,
    lon_col: str = 'lon_jgd',
    lat_col: str = 'lat_jgd',
    df_crs: str = 'EPSG:4612',
    proj_crs: str = 'EPSG:3857',  # m距離用（全国一律の近似）
    type_col: str = 'P04_001',     # 1=病院,2=診療所,3=歯科
    beds_col: str = 'P04_008',     # 病床数（病院のみ想定）
    emergency_col: str = 'P04_009',# 救急告示: 指定あり=1, 指定なし=9（想定）
    disaster_col: str = 'P04_010', # 災害拠点: 基幹=1, 地域=2, 指定なし=9（想定）
    use_types: tuple[int, ...] = (1, 2),  # まずは病院・診療所
    # bin しきい値
    hospital_threshold_m: int = 1000,
    clinic_threshold_m: int = 500,
    # 欠損距離の埋め（通常起きないはず）
    fill_missing_distance_m: float = 10_000.0,
    add_log_features: bool = True,
    add_nearest_attributes: bool = True,
) -> pd.DataFrame:
    """
    医療機関データ（点）から、物件点への最近傍距離を算出し特徴量追加。
    - 病院(1)と診療所(2)をデフォルトで利用（歯科(3)は原則後回し）
    - 距離は投影してm単位で計算
    - 任意で「最寄り病院の病床数」「救急/災害拠点フラグ」も付与可能

    追加される列（use_typesにより変動）:
      病院:
        dist_hospital_m, hospital_1km, dist_hospital_log
        (optional) hospital_beds_nearest, hospital_is_emergency_nearest, hospital_is_disaster_nearest
      診療所:
        dist_clinic_m, clinic_500m, dist_clinic_log
    """
    out = df.copy()

    # --- 入力チェック ---
    if lon_col not in out.columns or lat_col not in out.columns:
        raise ValueError(f'df に {lon_col}/{lat_col} がありません。')
    if medical_gdf is None or len(medical_gdf) == 0:
        raise ValueError('medical_gdf が空です。')
    if medical_gdf.crs is None:
        raise ValueError('medical_gdf.crs が None です。')

    # --- 物件点 ---
    gdf_pts = gpd.GeoDataFrame(
        out[[lon_col, lat_col]].copy(),
        geometry=gpd.points_from_xy(out[lon_col], out[lat_col]),
        crs=df_crs,
    )
    gdf_pts['pt_id'] = np.arange(len(gdf_pts), dtype=np.int64)

    # --- 医療データ前処理（タイプ絞り） ---
    med = medical_gdf.copy()
    med[type_col] = pd.to_numeric(med[type_col], errors='coerce')

    med = med.loc[med[type_col].isin(use_types)].copy()
    if len(med) == 0:
        raise ValueError(f'use_types={use_types} の条件で医療機関が0件になりました。')

    # --- 投影して距離(m) ---
    pts_proj = gdf_pts.to_crs(proj_crs)
    med_proj = med.to_crs(proj_crs)

    # --- タイプ別に最近傍 ---
    feat = pd.DataFrame({'pt_id': np.arange(len(out), dtype=np.int64)})

    def _nearest_for_type(type_code: int, dist_col_out: str, take_attrs: bool):
        m_sub = med_proj.loc[med_proj[type_col] == type_code].copy()
        if len(m_sub) == 0:
            # 施設が存在しない場合は遠距離で埋める
            feat[dist_col_out] = fill_missing_distance_m
            return None

        # 必要な属性列だけ保持（重い列は落とす）
        keep = ['geometry']
        if take_attrs:
            for c in [beds_col, emergency_col, disaster_col]:
                if c in m_sub.columns:
                    keep.append(c)
        m_sub = m_sub[keep].copy()

        j = gpd.sjoin_nearest(
            pts_proj[['pt_id', 'geometry']],
            m_sub,
            how='left',
            distance_col=dist_col_out,
        )
        # 代表行（pt_idごとに1行）
        j = j.drop_duplicates('pt_id').set_index('pt_id')

        feat[dist_col_out] = pd.to_numeric(j[dist_col_out], errors='coerce')

        return j

    # 病院(1)
    j_hosp = None
    if 1 in use_types:
        j_hosp = _nearest_for_type(1, 'dist_hospital_m', add_nearest_attributes)

    # 診療所(2)
    j_clinic = None
    if 2 in use_types:
        j_clinic = _nearest_for_type(2, 'dist_clinic_m', False)

    # 欠損埋め
    for c in ['dist_hospital_m', 'dist_clinic_m']:
        if c in feat.columns:
            feat[c] = feat[c].fillna(fill_missing_distance_m)

    # --- bin ---
    if 'dist_hospital_m' in feat.columns:
        feat['hospital_1km'] = (feat['dist_hospital_m'] <= hospital_threshold_m).astype('int8')
        if add_log_features:
            feat['dist_hospital_log'] = np.log1p(feat['dist_hospital_m'])

    if 'dist_clinic_m' in feat.columns:
        feat['clinic_500m'] = (feat['dist_clinic_m'] <= clinic_threshold_m).astype('int8')
        if add_log_features:
            feat['dist_clinic_log'] = np.log1p(feat['dist_clinic_m'])

    # --- 最寄り病院の属性（任意） ---
    if add_nearest_attributes and (j_hosp is not None):
        # 病床数
        if beds_col in j_hosp.columns:
            feat['hospital_beds_nearest'] = pd.to_numeric(j_hosp[beds_col], errors='coerce').fillna(0.0)

        # 救急告示（指定あり=1、それ以外=0）
        if emergency_col in j_hosp.columns:
            e = pd.to_numeric(j_hosp[emergency_col], errors='coerce')
            feat['hospital_is_emergency_nearest'] = (e == 1).astype('int8')

        # 災害拠点（基幹=1,地域=2 を 1扱い、それ以外0）
        if disaster_col in j_hosp.columns:
            d = pd.to_numeric(j_hosp[disaster_col], errors='coerce')
            feat['hospital_is_disaster_nearest'] = (d.isin([1, 2])).astype('int8')

    feat = feat.drop(columns=['pt_id'])

    out2 = out.reset_index(drop=True).join(feat, how='left')
    out2.index = out.index
    return out2


In [62]:
train_df_geo = add_medical_features_from_gdf(train_df_geo, hospital_df, use_types=(1, 2))
test_df_geo  = add_medical_features_from_gdf(test_df_geo,  hospital_df, use_types=(1, 2))


In [63]:
del hospital_df
gc.collect()

0

## 洪水浸水

#### 内水

In [64]:
ame_path = gis_path + '洪水浸水/ame.parquet'

In [65]:
ame_df = gpd.read_parquet(ame_path)

In [66]:
def _parse_depth_upper_m(s: pd.Series) -> pd.Series:
    """
    A51_005 のような文字列から「浸水深の上限(m)」を作る。
    例:
      '0.3m未満' -> 0.3
      '0.5m以上1m未満' -> 1.0
      '1m以上3m未満' -> 3.0
      '3m以上5m未満' -> 5.0
      '5m以上10m未満' -> 10.0
      '10m以上20m未満' -> 20.0
      '20m以上' -> 30.0 (上限が無いので便宜上 30m に丸め)
    """
    ss = s.astype('string').fillna('')

    # 'Xm未満' -> X
    m1 = ss.str.extract(r'^\s*([0-9.]+)\s*m\s*未満\s*$')[0]

    # 'A以上B未満' -> B
    m2 = ss.str.extract(r'^\s*[0-9.]+\s*m\s*以上\s*([0-9.]+)\s*m\s*未満\s*$')[0]

    # 'A以上' -> 便宜上 30m
    m3 = ss.str.contains(r'以上\s*$', na=False)

    out = pd.to_numeric(m1, errors='coerce')
    out = out.fillna(pd.to_numeric(m2, errors='coerce'))
    out = out.astype('float32')

    out = out.where(~m3, 30.0).astype('float32')
    out = out.where(out > 0, np.nan).astype('float32')
    return out

def _depth_rank_from_upper(upper: pd.Series) -> pd.Series:
    """
    上限(m)をランク整数に（単調増加なら何でもよい）
    """
    # 0.3, 1, 3, 5, 10, 20, 30 を想定
    bins = [0, 0.3, 1, 3, 5, 10, 20, 30, np.inf]
    # 0.3->1, 1->2, ... のような順位にする
    r = pd.cut(upper, bins=bins, labels=False, include_lowest=True)
    return r.astype('float32')

def add_ame_features(prop_df: pd.DataFrame, ame_poly: gpd.GeoDataFrame,
                     lon_col: str = 'lon_jgd', lat_col: str = 'lat_jgd',
                     prefix: str = 'ame', work_crs: str = 'EPSG:6677',
                     max_dist_m: float = 10000.0) -> pd.DataFrame:
    """
    雨水出水(内水)ポリゴンを物件(point)に突合して特徴量を返す。
    """
    # 物件点
    prop_g = gpd.GeoDataFrame(
        prop_df[[lon_col, lat_col]].copy(),
        geometry=gpd.points_from_xy(prop_df[lon_col], prop_df[lat_col]),
        crs='EPSG:4326'
    ).to_crs(work_crs)

    a = ame_poly[['A51_005', 'geometry']].copy()
    a = a.to_crs(work_crs)
    a['depth_upper_m'] = _parse_depth_upper_m(a['A51_005'])
    a['depth_rank'] = _depth_rank_from_upper(a['depth_upper_m'])

    # intersects（複数ヒット想定）
    j = prop_g[['geometry']].sjoin(a[['depth_upper_m', 'depth_rank', 'geometry']],
                                  how='left', predicate='intersects')

    out = pd.DataFrame(index=prop_df.index)

    out[f'{prefix}_in'] = (
        j['index_right'].notna()
        .groupby(level=0).max()
        .reindex(prop_df.index, fill_value=False)
        .astype('int8')
    )

    out[f'{prefix}_depth_max_m'] = (
        pd.to_numeric(j['depth_upper_m'], errors='coerce')
        .groupby(level=0).max()
        .reindex(prop_df.index)
        .astype('float32')
    )

    out[f'{prefix}_depth_rank_max'] = (
        pd.to_numeric(j['depth_rank'], errors='coerce')
        .groupby(level=0).max()
        .reindex(prop_df.index)
        .astype('float32')
    )

    # 最近傍距離（複数行→minで1行化）
    near = prop_g[['geometry']].sjoin_nearest(
        a[['geometry']],
        how='left',
        max_distance=max_dist_m,
        distance_col=f'{prefix}_dist_m'
    )
    out[f'{prefix}_dist_m'] = (
        near[f'{prefix}_dist_m']
        .groupby(level=0).min()
        .reindex(prop_df.index)
        .astype('float32')
    )

    # 追加の安定特徴
    out[f'{prefix}_depth_max_log1p'] = np.log1p(out[f'{prefix}_depth_max_m']).astype('float32')

    out[f'{prefix}_depth_ge1m'] = (out[f'{prefix}_depth_max_m'] >= 1.0).astype('int8')
    out[f'{prefix}_depth_ge3m'] = (out[f'{prefix}_depth_max_m'] >= 3.0).astype('int8')

    # 10km以上は同一視してログ変換
    out['ame_dist_m_clip'] = out['ame_dist_m'].clip(upper=10000)
    out['ame_dist_log'] = np.log1p(out['ame_dist_m_clip'])

    out['ame_depth_effect'] = (
        out['ame_depth_max_log1p']
        .where(out['ame_dist_m'] <= 2000, 0.0)
    )

    return out


In [67]:
train_ame = add_ame_features(train_df_geo, ame_df, prefix='ame', max_dist_m=10000.0)
test_ame  = add_ame_features(test_df_geo,  ame_df, prefix='ame', max_dist_m=10000.0)

train_df_geo = train_df_geo.join(train_ame)
test_df_geo  = test_df_geo.join(test_ame)

In [68]:
print('ame_in hit rate train:', train_df_geo['ame_in'].mean())
print('ame_in hit rate test :', test_df_geo['ame_in'].mean())

ame_in hit rate train: 0.0007501566261087479
ame_in hit rate test : 0.0007204034259185143


#### 外水

In [69]:
in_dir = Path(gis_path + '洪水浸水/1次メッシュ')
keikaku_files = sorted(in_dir.glob('*/10_計画規模/*.geojson'))
saidai_files = sorted(in_dir.glob('*/20_想定最大規模/*.geojson'))

In [70]:
from __future__ import annotations

import gc
import re
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd


def _extract_region_id(fp: str | Path) -> str:
    name = Path(fp).name
    m = re.search(r'_(\d{4})\.geojson$', name)
    if not m:
        raise ValueError(f'Unexpected filename format: {name}')
    return m.group(1)


def _pair_files_by_region(files: list[str | Path]) -> dict[str, Path]:
    out: dict[str, Path] = {}
    for fp in files:
        rid = _extract_region_id(fp)
        out[rid] = Path(fp)
    return out


def _read_geojson_mincols(fp: Path, cols: list[str]) -> gpd.GeoDataFrame:
    gdf = gpd.read_file(fp, engine='pyogrio')
    keep = [c for c in cols if c in gdf.columns] + ['geometry']
    return gdf[keep]


def _agg_rank_and_in(joined: gpd.GeoDataFrame, prop_index: pd.Index, rank_col: str) -> tuple[pd.Series, pd.Series]:
    in_flag = (
        joined['index_right'].notna()
        .groupby(level=0).max()
        .reindex(prop_index, fill_value=False)
        .astype('int8')
    )
    rank_max = (
        pd.to_numeric(joined[rank_col], errors='coerce')
        .groupby(level=0).max()
        .reindex(prop_index)
        .astype('float32')
    )
    return in_flag, rank_max


def _nearest_dist_m(
    prop_m: gpd.GeoDataFrame,
    poly_m: gpd.GeoDataFrame,
    prop_index: pd.Index,
    max_dist_m: float,
    dist_name: str
) -> pd.Series:
    near = prop_m[['geometry']].sjoin_nearest(
        poly_m[['geometry']],
        how='left',
        max_distance=max_dist_m,
        distance_col=dist_name,
    )
    dist = (
        near[dist_name]
        .groupby(level=0).min()
        .reindex(prop_index)
        .astype('float32')
    )
    return dist


def add_kouzui_keikaku_saidai_features_stream(
    prop_df: pd.DataFrame,
    keikaku_files: list[str | Path],
    saidai_files: list[str | Path],
    lon_col: str = 'lon_jgd',
    lat_col: str = 'lat_jgd',
    src_crs: str = 'EPSG:6668',
    work_crs_m: str = 'EPSG:6677',
    max_dist_m: float = 10000.0,
    prefix: str = 'kouzui',
    verbose: bool = True,
) -> pd.DataFrame:

    prop_g = gpd.GeoDataFrame(
        prop_df[[lon_col, lat_col]].copy(),
        geometry=gpd.points_from_xy(prop_df[lon_col], prop_df[lat_col]),
        crs='EPSG:4326',
        index=prop_df.index,
    ).to_crs(src_crs)

    out = pd.DataFrame(index=prop_df.index)
    out[f'{prefix}_keikaku_in'] = np.int8(0)
    out[f'{prefix}_keikaku_rank_max'] = np.float32(np.nan)
    out[f'{prefix}_keikaku_dist_m'] = np.float32(np.nan)

    out[f'{prefix}_saidai_in'] = np.int8(0)
    out[f'{prefix}_saidai_rank_max'] = np.float32(np.nan)
    out[f'{prefix}_saidai_dist_m'] = np.float32(np.nan)

    keikaku_map = _pair_files_by_region(keikaku_files)
    saidai_map = _pair_files_by_region(saidai_files)
    region_ids = sorted(set(keikaku_map.keys()) | set(saidai_map.keys()))

    if verbose:
        print(f'[{prefix}] regions={len(region_ids)} (keikaku={len(keikaku_map)}, saidai={len(saidai_map)})')

    for i, rid in enumerate(region_ids, 1):
        fp_k = keikaku_map.get(rid)
        fp_s = saidai_map.get(rid)
        if fp_k is None and fp_s is None:
            continue

        k_gdf = None
        s_gdf = None

        try:
            if fp_k is not None:
                k_gdf = _read_geojson_mincols(fp_k, cols=['A31b_101'])
                if k_gdf.crs is None:
                    k_gdf = k_gdf.set_crs(src_crs)
                else:
                    k_gdf = k_gdf.to_crs(src_crs)

            if fp_s is not None:
                s_gdf = _read_geojson_mincols(fp_s, cols=['A31b_201'])
                if s_gdf.crs is None:
                    s_gdf = s_gdf.set_crs(src_crs)
                else:
                    s_gdf = s_gdf.to_crs(src_crs)

            bounds_list = []
            if k_gdf is not None and len(k_gdf) > 0:
                bounds_list.append(k_gdf.total_bounds)
            if s_gdf is not None and len(s_gdf) > 0:
                bounds_list.append(s_gdf.total_bounds)
            if not bounds_list:
                continue

            minx = min(b[0] for b in bounds_list)
            miny = min(b[1] for b in bounds_list)
            maxx = max(b[2] for b in bounds_list)
            maxy = max(b[3] for b in bounds_list)

            cand_mask = (
                (prop_g.geometry.x >= minx) & (prop_g.geometry.x <= maxx) &
                (prop_g.geometry.y >= miny) & (prop_g.geometry.y <= maxy)
            )
            cand_idx = prop_g.index[cand_mask]
            if len(cand_idx) == 0:
                continue

            prop_sub = prop_g.loc[cand_idx, ['geometry']]
            prop_sub_m = prop_sub.to_crs(work_crs_m)

            # 計画規模
            if k_gdf is not None and len(k_gdf) > 0 and 'A31b_101' in k_gdf.columns:
                k_m = k_gdf[['A31b_101', 'geometry']].copy().to_crs(work_crs_m)

                j_k = prop_sub_m.sjoin(k_m, how='left', predicate='intersects')
                in_k, rank_k = _agg_rank_and_in(j_k, prop_sub_m.index, rank_col='A31b_101')
                dist_k = _nearest_dist_m(
                    prop_sub_m, k_m, prop_sub_m.index,
                    max_dist_m, dist_name=f'{prefix}_keikaku_dist_m'
                )

                out.loc[cand_idx, f'{prefix}_keikaku_in'] = in_k.values
                out.loc[cand_idx, f'{prefix}_keikaku_rank_max'] = rank_k.values
                out.loc[cand_idx, f'{prefix}_keikaku_dist_m'] = dist_k.values

            # 最大規模
            if s_gdf is not None and len(s_gdf) > 0 and 'A31b_201' in s_gdf.columns:
                s_m = s_gdf[['A31b_201', 'geometry']].copy().to_crs(work_crs_m)

                j_s = prop_sub_m.sjoin(s_m, how='left', predicate='intersects')
                in_s, rank_s = _agg_rank_and_in(j_s, prop_sub_m.index, rank_col='A31b_201')
                dist_s = _nearest_dist_m(
                    prop_sub_m, s_m, prop_sub_m.index,
                    max_dist_m, dist_name=f'{prefix}_saidai_dist_m'
                )

                out.loc[cand_idx, f'{prefix}_saidai_in'] = in_s.values
                out.loc[cand_idx, f'{prefix}_saidai_rank_max'] = rank_s.values
                out.loc[cand_idx, f'{prefix}_saidai_dist_m'] = dist_s.values

        finally:
            del k_gdf, s_gdf
            gc.collect()

        if verbose and (i % 50 == 0 or i == len(region_ids)):
            print(f'[{prefix}] processed {i}/{len(region_ids)} regions')

    # 型整形
    out[f'{prefix}_keikaku_in'] = out[f'{prefix}_keikaku_in'].astype('int8')
    out[f'{prefix}_saidai_in'] = out[f'{prefix}_saidai_in'].astype('int8')

    out[f'{prefix}_keikaku_rank_max'] = out[f'{prefix}_keikaku_rank_max'].astype('float32')
    out[f'{prefix}_saidai_rank_max'] = out[f'{prefix}_saidai_rank_max'].astype('float32')

    # --- dist: clip + log1p を標準装備にする（ここが追加） ---
    for col in [f'{prefix}_keikaku_dist_m', f'{prefix}_saidai_dist_m']:
        s = pd.to_numeric(out[col], errors='coerce').astype('float32')
        s = s.fillna(np.float32(max_dist_m))
        s = s.clip(lower=np.float32(0.0), upper=np.float32(max_dist_m)).astype('float32')
        out[col] = s
        out[f'{col}_log1p'] = np.log1p(s).astype('float32')

    # rank>=3 フラグ
    out[f'{prefix}_keikaku_rank_ge3'] = (out[f'{prefix}_keikaku_rank_max'] >= 3).astype('int8')
    out[f'{prefix}_saidai_rank_ge3'] = (out[f'{prefix}_saidai_rank_max'] >= 3).astype('int8')

    return out


In [71]:
train_kouzui = add_kouzui_keikaku_saidai_features_stream(
    prop_df=train_df_geo,                  # lon_jgd/lat_jgd を持つDF
    keikaku_files=keikaku_files,        # Pathのlist
    saidai_files=saidai_files,          # Pathのlist
    lon_col='lon_jgd',
    lat_col='lat_jgd',
    max_dist_m=10000.0,
    prefix='kouzui',
    verbose=True,
)

test_kouzui = add_kouzui_keikaku_saidai_features_stream(
    prop_df=test_df_geo,
    keikaku_files=keikaku_files,
    saidai_files=saidai_files,
    lon_col='lon_jgd',
    lat_col='lat_jgd',
    max_dist_m=10000.0,
    prefix='kouzui',
    verbose=True,
)

train_df_geo = pd.concat([train_df_geo, train_kouzui], axis=1)
test_df_geo = pd.concat([test_df_geo, test_kouzui], axis=1)


[kouzui] regions=105 (keikaku=101, saidai=105)
[kouzui] processed 50/105 regions
[kouzui] regions=105 (keikaku=101, saidai=105)
[kouzui] processed 50/105 regions


In [72]:
print('kouzui_keikaku_in hit rate train:', train_df_geo['kouzui_keikaku_in'].mean())
print('kouzui_keikaku_in hit rate test :', test_df_geo['kouzui_keikaku_in'].mean())

print('kouzui_saidai_in hit rate train:', train_df_geo['kouzui_saidai_in'].mean())
print('kouzui_saidai_in hit rate test :', test_df_geo['kouzui_saidai_in'].mean())

kouzui_keikaku_in hit rate train: 0.042077466723821455
kouzui_keikaku_in hit rate test : 0.0449407223600772
kouzui_saidai_in hit rate train: 0.08733691649904925
kouzui_saidai_in hit rate test : 0.0898814447201544


## 土砂災害

In [73]:
kyuusha_path = gis_path + '土砂災害/kyuusha.parquet'
doshamesh_path = gis_path + '土砂災害/doshamesh.parquet'
doshakeikai_path = gis_path + '土砂災害/doshakeikai.parquet'

In [74]:
kyuusha_df = gpd.read_parquet(kyuusha_path)
doshamesh_df = gpd.read_parquet(doshamesh_path)
doshakeikai_df = gpd.read_parquet(doshakeikai_path)

In [75]:
import os
from pathlib import Path

# GeoJSONが巨大なときの制限解除（必要なら）
os.environ.setdefault('OGR_GEOJSON_MAX_OBJ_SIZE', '0')

def _ensure_cols(df, cols, name='df'):
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f'{name}: missing columns: {missing}')

def _to_points_gdf(df: pd.DataFrame, lon_col: str, lat_col: str, crs='EPSG:4326') -> gpd.GeoDataFrame:
    _ensure_cols(df, [lon_col, lat_col], name='properties')
    gdf = gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df[lon_col].astype(float), df[lat_col].astype(float)),
        crs=crs,
    )
    return gdf

def _to_crs_safe(gdf: gpd.GeoDataFrame, crs: str) -> gpd.GeoDataFrame:
    if gdf.crs is None:
        return gdf.set_crs(crs)
    else:
        return gdf.to_crs(crs)

def _parse_mm_per(s: pd.Series) -> pd.Series:
    # '74mm/24hr' '38mm/hr' などを float に
    s = s.astype('string')
    s = s.str.replace('mm/24hr', '', regex=False).str.replace('mm/hr', '', regex=False)
    s = s.str.replace('mm', '', regex=False).str.strip()
    return pd.to_numeric(s, errors='coerce')

def _parse_m3(s: pd.Series) -> pd.Series:
    s = s.astype('string').str.replace('m3', '', regex=False).str.strip()
    return pd.to_numeric(s, errors='coerce')

def _parse_m(s: pd.Series) -> pd.Series:
    s = s.astype('string').str.replace('m', '', regex=False).str.strip()
    return pd.to_numeric(s, errors='coerce')


In [76]:
# 物件
train_prop = _to_points_gdf(train_df_geo, 'lon_jgd', 'lat_jgd', crs='EPSG:4326')
test_prop  = _to_points_gdf(test_df_geo,  'lon_jgd', 'lat_jgd', crs='EPSG:4326')

# まず4326へ統一
kyuusha_4326      = _to_crs_safe(kyuusha_df, 'EPSG:4326')
dosham_mesh_4326  = _to_crs_safe(doshamesh_df, 'EPSG:4326')
doshakeikai_4326  = _to_crs_safe(doshakeikai_df, 'EPSG:4326')

# 距離用（メートル系）
train_m = train_prop.to_crs('EPSG:3857')
test_m  = test_prop.to_crs('EPSG:3857')

kyuusha_m     = kyuusha_4326.to_crs('EPSG:3857')
mesh_m        = dosham_mesh_4326.to_crs('EPSG:3857')
keikai_m      = doshakeikai_4326.to_crs('EPSG:3857')

In [77]:
def add_polygon_hit_and_dist(
    prop_m: gpd.GeoDataFrame,
    poly_m: gpd.GeoDataFrame,
    prefix: str,
    max_dist_m: float = 5000.0
) -> pd.DataFrame:
    # hit（intersects）: 1物件が複数ヒットするので groupby(max) で集約
    hit = prop_m[['geometry']].sjoin(poly_m[['geometry']], how='left', predicate='intersects')
    in_flag = (
        hit['index_right']
        .notna()
        .groupby(level=0)
        .max()
        .reindex(prop_m.index, fill_value=False)
        .astype('int8')
        .rename(f'{prefix}_in')
    )

    # nearest（距離）: タイ等で複数行になることがあるので groupby(min) で集約
    near = prop_m[['geometry']].sjoin_nearest(
        poly_m[['geometry']],
        how='left',
        max_distance=max_dist_m,
        distance_col=f'{prefix}_dist_m'
    )
    dist = (
        near[f'{prefix}_dist_m']
        .groupby(level=0)
        .min()
        .reindex(prop_m.index)
        .astype('float32')
        .rename(f'{prefix}_dist_m')
    )

    out = pd.concat([in_flag, dist], axis=1)
    return out


train_kyuusha = add_polygon_hit_and_dist(train_m, kyuusha_m, prefix='kyuusha', max_dist_m=10000.0)
test_kyuusha  = add_polygon_hit_and_dist(test_m,  kyuusha_m, prefix='kyuusha', max_dist_m=10000.0)

In [78]:
# genshou_code: 1=急傾斜地の崩壊, 2=土石流, 3=地滑り
# kuiki_code: 1=警戒(指定済), 2=特別警戒(指定済), 3=警戒(指定前), 4=特別警戒(指定前)

def add_keikai_features(
    prop_m: gpd.GeoDataFrame,
    keikai_m: gpd.GeoDataFrame,
    target_ym_series: pd.Series,          # 物件側のYYYYMM（indexはprop_m.indexと同じ）
    max_dist_m: float = 5000.0
) -> pd.DataFrame:
    # fail-fast
    if target_ym_series is None:
        raise ValueError('target_ym_series is required for kouji_date leak control')
    if not target_ym_series.index.equals(prop_m.index):
        # 同じ並びでなくてもreindexで揃える
        target_ym_series = target_ym_series.reindex(prop_m.index)

    k = keikai_m[['genshou_code', 'kuiki_code', 'kouji_date', 'geometry']].copy()

    # intersects（複数ヒット想定）
    j = prop_m[['geometry']].sjoin(k, how='left', predicate='intersects')

    def _flag(col, val, name):
        return (
            (j[col] == val)
            .groupby(level=0)
            .max()
            .reindex(prop_m.index, fill_value=False)
            .astype('int8')
            .rename(name)
        )

    # 現象別
    f_g1 = _flag('genshou_code', 1, 'keikai_genshou1_in')
    f_g2 = _flag('genshou_code', 2, 'keikai_genshou2_in')
    f_g3 = _flag('genshou_code', 3, 'keikai_genshou3_in')

    # 区域別
    f_k1 = _flag('kuiki_code', 1, 'keikai_kuiki1_in')
    f_k2 = _flag('kuiki_code', 2, 'keikai_kuiki2_in')
    f_k3 = _flag('kuiki_code', 3, 'keikai_kuiki3_in')
    f_k4 = _flag('kuiki_code', 4, 'keikai_kuiki4_in')

    # =========================
    # 公示日（最小）with leak control
    # =========================
    kd = pd.to_datetime(j['kouji_date'], errors='coerce')

    # target_ym（YYYYMM）→ datetime（当月1日）に変換（prop indexへ付与）
    ym = target_ym_series.astype('float64')
    y = np.floor(ym / 100.0)
    m = ym - y * 100.0
    base = pd.to_datetime(
        dict(year=y, month=m, day=np.ones_like(y)),
        errors='coerce'
    )

    # j（多重ヒット）の index（=prop index）に合わせて base を展開
    base_for_j = base.reindex(j.index)

    # 未来の公示日を無効化（NaT）
    kd = kd.where((kd.isna()) | (base_for_j.isna()) | (kd <= base_for_j), pd.NaT)

    # 未来無効化後の最小公示日
    kouji_min = kd.groupby(level=0).min().reindex(prop_m.index).rename('keikai_kouji_date_min')

    # 派生：経過年数（取引日 - 公示日）
    kouji_age_year = ((base - kouji_min).dt.days / 365.25).astype('float32').rename('keikai_kouji_age_year')

    # nearest距離（タイ等で複数行→groupby(min)で1行化）
    near = prop_m[['geometry']].sjoin_nearest(
        k[['geometry']],
        how='left',
        max_distance=max_dist_m,
        distance_col='keikai_dist_m'
    )
    dist = (
        near['keikai_dist_m']
        .groupby(level=0)
        .min()
        .reindex(prop_m.index)
        .astype('float32')
        .rename('keikai_dist_m')
    )

    out = pd.concat(
        [f_g1, f_g2, f_g3, f_k1, f_k2, f_k3, f_k4, kouji_min, kouji_age_year, dist],
        axis=1
    )
    return out

train_keikai = add_keikai_features(
    train_m, keikai_m,
    target_ym_series=train_df_geo.loc[train_m.index, 'target_ym'],
    max_dist_m=10000.0
)

test_keikai = add_keikai_features(
    test_m, keikai_m,
    target_ym_series=test_df_geo.loc[test_m.index, 'target_ym'],
    max_dist_m=10000.0
)

In [79]:
mesh = mesh_m.copy()

# 災害種別（A30a5_004）を保持
# 数値項目をfloat化（存在する列だけ）
for c in ['A30a5_005', 'A30a5_006']:
    if c in mesh.columns:
        mesh[c] = _parse_mm_per(mesh[c])

if 'A30a5_008' in mesh.columns:
    mesh['A30a5_008'] = _parse_m3(mesh['A30a5_008'])
if 'A30a5_009' in mesh.columns:
    mesh['A30a5_009'] = _parse_m(mesh['A30a5_009'])

def add_mesh_features(prop_m: gpd.GeoDataFrame, mesh_m: gpd.GeoDataFrame, prefix='dosham') -> pd.DataFrame:
    cols = [c for c in ['A30a5_004','A30a5_005','A30a5_006','A30a5_007','A30a5_008','A30a5_009','A30a5_010'] if c in mesh_m.columns]
    j = prop_m[['geometry']].sjoin(mesh_m[cols + ['geometry']], how='left', predicate='intersects')

    # 物件ごとに集約
    out = pd.DataFrame(index=prop_m.index)

    # inフラグ（メッシュが存在）
    out[f'{prefix}_in'] = j['index_right'].notna().groupby(level=0).max().fillna(False).astype('int8')

    # 災害種別（A30a5_004）が列挙型なら one-hot 的に持つ（代表的な値を見て調整）
    if 'A30a5_004' in j.columns:
        # 例：'がけ崩れ','土石流','地すべり','雪崩(表層)' などが入り得る
        phen = j['A30a5_004'].astype('string')
        # よく出るカテゴリは実データで確認して追加してください
        for key, name in [
            ('がけ崩れ', f'{prefix}_phen_gake'),
            ('土石流', f'{prefix}_phen_doseki'),
            ('地すべり', f'{prefix}_phen_jisuberi'),
            ('雪崩', f'{prefix}_phen_nadare'),
        ]:
            out[name] = phen.str.contains(key, na=False).groupby(level=0).max().astype('int8')

    # 数値系：max（保守的に最大リスク寄り）
    for c in ['A30a5_005','A30a5_006','A30a5_007','A30a5_008','A30a5_009','A30a5_010']:
        if c in j.columns:
            out[f'{prefix}_{c}_max'] = pd.to_numeric(j[c], errors='coerce').groupby(level=0).max()

    return out.reindex(prop_m.index)

train_mesh = add_mesh_features(train_m, mesh, prefix='dosham')
test_mesh  = add_mesh_features(test_m,  mesh, prefix='dosham')


In [80]:
train_add = pd.concat([train_kyuusha, train_keikai, train_mesh], axis=1)
test_add  = pd.concat([test_kyuusha,  test_keikai,  test_mesh], axis=1)

train_df_geo = train_df_geo.join(train_add)
test_df_geo  = test_df_geo.join(test_add)

In [81]:
print('kyuusha_in hit rate train:', train_df_geo['kyuusha_in'].mean())
print('kyuusha_in hit rate test :', test_df_geo['kyuusha_in'].mean())

print('dosham_in hit rate train:', train_df_geo['dosham_in'].mean())
print('dosham_in hit rate test :', test_df_geo['dosham_in'].mean())

print('dosham_in hit rate train:', train_df_geo['dosham_in'].mean())
print('dosham_in hit rate test :', test_df_geo['dosham_in'].mean())

kyuusha_in hit rate train: 0.003514470054187138
kyuusha_in hit rate test : 0.0026681608367352385
dosham_in hit rate train: 0.197513766610611
dosham_in hit rate test : 0.19574517285235288
dosham_in hit rate train: 0.197513766610611
dosham_in hit rate test : 0.19574517285235288


## 地盤沈下

In [82]:
file = Path(gis_path + '地盤沈下地域/C35-60L-48-01.0a_GML/C35-60L-2K_SubsidenceDistrict.shp')
jiban_gdf = read_dataframe(file)

In [83]:
def prepare_jiban_lines(
    jiban_gdf: gpd.GeoDataFrame,
    src_crs: str = 'EPSG:4612',
    work_crs_m: str = 'EPSG:6677',
) -> gpd.GeoDataFrame:
    g = jiban_gdf
    if g.crs is None:
        g = g.set_crs(src_crs)
    else:
        g = g.to_crs(src_crs)

    # 必要列だけに絞る（メモリ・速度に効く）
    cols = [c for c in ['C35_002', 'C35_003'] if c in g.columns]
    g = g[cols + ['geometry']].copy()

    # 距離計算用CRSへ（ここを1回で済ませるのが重要）
    g = g.to_crs(work_crs_m)

    # sindex を事前に作っておく（環境によっては効く）
    _ = g.sindex
    return g


In [84]:
def _ensure_target_year(
    df: pd.DataFrame,
    target_ym_col: str = 'target_ym',
    target_year_col: str = 'target_year',
) -> pd.Series:
    """
    target_year があればそれを使用。
    なければ target_ym (YYYYMM想定) から年を抽出。
    どちらも無ければ全NaNを返す（リーク対策ができないため）。
    """
    if target_year_col in df.columns:
        y = pd.to_numeric(df[target_year_col], errors='coerce')
        return y.astype('float32')

    if target_ym_col in df.columns:
        v = df[target_ym_col]
        if np.issubdtype(v.dtype, np.number):
            y = (pd.to_numeric(v, errors='coerce') // 100).astype('float32')
            return y

        s = v.astype('string')
        y = pd.to_numeric(s.str.slice(0, 4), errors='coerce').astype('float32')
        return y

    return pd.Series(np.nan, index=df.index, dtype='float32')


In [ ]:
# def add_jiban_chinka_features_fast(
#     prop_df: pd.DataFrame,
#     jiban_m: gpd.GeoDataFrame,              # prepare済み（EPSG:6677）
#     lon_col: str = 'lon_jgd',
#     lat_col: str = 'lat_jgd',
#     target_ym_col: str = 'target_ym',
#     target_year_col: str = 'target_year',
#     prop_src_crs: str = 'EPSG:4326',
#     work_crs_m: str = 'EPSG:6677',
#     max_dist_m: float = 10000.0,
#     prefix: str = 'jiban',
# ) -> pd.DataFrame:
#     # 物件点 → m系
#     prop_g = gpd.GeoDataFrame(
#         prop_df[[lon_col, lat_col]].copy(),
#         geometry=gpd.points_from_xy(prop_df[lon_col], prop_df[lat_col]),
#         crs=prop_src_crs,
#         index=prop_df.index,
#     ).to_crs(work_crs_m)

#     # nearest（属性も取る）
#     need_cols = [c for c in ['C35_002', 'C35_003'] if c in jiban_m.columns]
#     near = prop_g[['geometry']].sjoin_nearest(
#         jiban_m[need_cols + ['geometry']],
#         how='left',
#         max_distance=max_dist_m,
#         distance_col=f'{prefix}_dist_m',
#     )

#     out = pd.DataFrame(index=prop_df.index)

#     # dist：minで一意化
#     dist = (
#         pd.to_numeric(near[f'{prefix}_dist_m'], errors='coerce')
#         .groupby(level=0).min()
#         .reindex(out.index)
#         .astype('float32')
#     )
#     out[f'{prefix}_dist_m'] = dist

#     # clip + log1p（標準装備）
#     out[f'{prefix}_dist_m_clip'] = out[f'{prefix}_dist_m'].clip(0.0, float(max_dist_m)).astype('float32')
#     out[f'{prefix}_dist_log1p'] = np.log1p(out[f'{prefix}_dist_m_clip']).astype('float32')

#     # 閾値フラグ（NaNはFalseにする）
#     out[f'{prefix}_near_100m'] = (out[f'{prefix}_dist_m'] <= 100.0).fillna(False).astype('int8')
#     out[f'{prefix}_near_300m'] = (out[f'{prefix}_dist_m'] <= 300.0).fillna(False).astype('int8')
#     out[f'{prefix}_near_1000m'] = (out[f'{prefix}_dist_m'] <= 1000.0).fillna(False).astype('int8')

#     # ========= nearest属性（NaN idx を除外して安全に拾う） =========
#     d = pd.to_numeric(near[f'{prefix}_dist_m'], errors='coerce')
#     idx_min = d.groupby(level=0).idxmin()          # 見つからない物件は NaN
#     idx_min_valid = idx_min.dropna().astype(int)   # ここがポイント（nan排除）

#     # nearest を “全indexぶん欠損で初期化”
#     year_near = pd.Series(np.nan, index=out.index, dtype='float32')
#     type_near = pd.Series(pd.NA, index=out.index, dtype='string')

#     if len(idx_min_valid) > 0:
#         nearest = near.loc[idx_min_valid.values, :].copy()
#         nearest.index = idx_min_valid.index  # index=prop_index

#         if 'C35_002' in nearest.columns:
#             year_near.loc[nearest.index] = pd.to_numeric(nearest['C35_002'], errors='coerce').astype('float32')

#         if 'C35_003' in nearest.columns:
#             type_near.loc[nearest.index] = nearest['C35_003'].astype('string')

#     # リーク対策
#     target_year = _ensure_target_year(prop_df, target_ym_col=target_ym_col, target_year_col=target_year_col)
#     is_future = year_near.notna() & target_year.notna() & (year_near > target_year)

#     year_near = year_near.mask(is_future, np.nan).astype('float32')
#     out[f'{prefix}_year_nearest'] = year_near
#     out[f'{prefix}_year_age'] = (target_year - out[f'{prefix}_year_nearest']).astype('float32')

#     # タイプone-hot（nearestのみ・未来は無効）
#     for code in ['02', '04', '70', '80', '90', '99']:
#         col = f'{prefix}_type_{code}_nearest'
#         flag = (type_near == code)
#         flag = flag.where(~is_future, False)  # 未来は無効
#         out[col] = flag.fillna(False).astype('int8')

#     return out

In [ ]:
# jiban_m = prepare_jiban_lines(jiban_gdf, src_crs='EPSG:4612', work_crs_m='EPSG:6677')

# train_jiban = add_jiban_chinka_features_fast(train_df_geo, jiban_m, prop_src_crs='EPSG:4326', prefix='jiban')
# test_jiban  = add_jiban_chinka_features_fast(test_df_geo,  jiban_m, prop_src_crs='EPSG:4326', prefix='jiban')

# train_df_geo = pd.concat([train_df_geo, train_jiban], axis=1)
# test_df_geo  = pd.concat([test_df_geo,  test_jiban], axis=1)

ValueError: Length mismatch: Expected axis has 41968885 elements, new values have 233031 elements

## 沿岸浸水

In [88]:
tsunami_path = gis_path + '津波浸水/tsunami.parquet'
takashio_path = gis_path + '高潮浸水/takashio.parquet'

In [89]:
tsunami_df = gpd.read_parquet(tsunami_path)
takashio_df = gpd.read_parquet(takashio_path)

In [90]:
def _parse_depth_upper_m_generic(s: pd.Series) -> pd.Series:
    """
    浸水深の区分文字列から「上限(m)」を抽出する（A40_003 / A49_003想定）。
    例:
      '0.3m未満' -> 0.3
      '0.01m以上〜0.3m未満' -> 0.3
      '10.0m以上〜20.0m未満' -> 20.0
      '20m以上' -> 30.0（上限なしは便宜上30m）
    """
    ss = s.astype('string').fillna('')

    # 波ダッシュ等を正規化
    ss = ss.str.replace('～', '〜', regex=False)

    # 'Xm未満' -> X
    m1 = ss.str.extract(r'^\s*([0-9.]+)\s*m\s*未満\s*$', expand=False)

    # 'A以上〜B未満' / 'A以上～B未満' -> B
    m2 = ss.str.extract(
        r'^\s*[0-9.]+\s*m\s*以上\s*[〜\-]\s*([0-9.]+)\s*m\s*未満\s*$',
        expand=False
    )

    # 'A以上B未満'（区切り記号なしも一応） -> B
    m2b = ss.str.extract(
        r'^\s*[0-9.]+\s*m\s*以上\s*([0-9.]+)\s*m\s*未満\s*$',
        expand=False
    )

    # 'A以上' -> 便宜上 30m
    m3 = ss.str.contains(r'以上\s*$', na=False)

    out = pd.to_numeric(m1, errors='coerce')
    out = out.fillna(pd.to_numeric(m2, errors='coerce'))
    out = out.fillna(pd.to_numeric(m2b, errors='coerce'))
    out = out.astype('float32')

    out = out.where(~m3, 30.0).astype('float32')
    out = out.where(out > 0, np.nan).astype('float32')
    return out

def _depth_rank_from_upper(upper: pd.Series) -> pd.Series:
    """
    上限(m)を単調なランクへ（津波/高潮とも同じ思想でOK）
    """
    bins = [0, 0.01, 0.3, 0.5, 1, 3, 5, 10, 20, 30, np.inf]
    r = pd.cut(upper, bins=bins, labels=False, include_lowest=True)
    return r.astype('float32')

def add_inundation_polygon_features(
    prop_df: pd.DataFrame,
    poly_gdf: gpd.GeoDataFrame,
    depth_col: str,
    lon_col: str = 'lon_jgd',
    lat_col: str = 'lat_jgd',
    prefix: str = 'inund',
    prop_input_crs: str = 'EPSG:4326',
    poly_input_crs: str | None = None,
    work_crs: str = 'EPSG:6677',   # 距離用（メートル）
    max_dist_m: float = 10000.0
) -> pd.DataFrame:
    """
    浸水ポリゴン（津波/高潮）を物件Pointに突合し、深さカテゴリ由来の特徴量 + 最近傍距離を返す。
    """

    # 物件点
    prop_g = gpd.GeoDataFrame(
        prop_df[[lon_col, lat_col]].copy(),
        geometry=gpd.points_from_xy(prop_df[lon_col], prop_df[lat_col]),
        crs=prop_input_crs
    )

    # ポリゴン側（CRSが付いていない/怪しい場合の保険）
    a = poly_gdf[[depth_col, 'geometry']].copy()
    if poly_input_crs is not None:
        a = a.set_crs(poly_input_crs, allow_override=True)

    # 距離計算・sjoin安定化のためメートル系へ
    prop_m = prop_g.to_crs(work_crs)
    a_m = a.to_crs(work_crs)

    a_m['depth_upper_m'] = _parse_depth_upper_m_generic(a_m[depth_col])
    a_m['depth_rank'] = _depth_rank_from_upper(a_m['depth_upper_m'])

    # intersects（複数ヒット前提 -> groupby(level=0) で 1物件1行へ）
    j = prop_m[['geometry']].sjoin(
        a_m[['depth_upper_m', 'depth_rank', 'geometry']],
        how='left',
        predicate='intersects'
    )

    out = pd.DataFrame(index=prop_df.index)

    out[f'{prefix}_in'] = (
        j['index_right'].notna()
        .groupby(level=0).max()
        .reindex(prop_df.index, fill_value=False)
        .astype('int8')
    )

    out[f'{prefix}_depth_max_m'] = (
        pd.to_numeric(j['depth_upper_m'], errors='coerce')
        .groupby(level=0).max()
        .reindex(prop_df.index)
        .astype('float32')
    )

    out[f'{prefix}_depth_rank_max'] = (
        pd.to_numeric(j['depth_rank'], errors='coerce')
        .groupby(level=0).max()
        .reindex(prop_df.index)
        .astype('float32')
    )

    # 最近傍距離（複数行->minで1行化）
    near = prop_m[['geometry']].sjoin_nearest(
        a_m[['geometry']],
        how='left',
        max_distance=max_dist_m,
        distance_col=f'{prefix}_dist_m'
    )

    out[f'{prefix}_dist_m'] = (
        near[f'{prefix}_dist_m']
        .groupby(level=0).min()
        .reindex(prop_df.index)
        .astype('float32')
    )

    # 追加の安定特徴
    out[f'{prefix}_depth_max_log1p'] = np.log1p(out[f'{prefix}_depth_max_m']).astype('float32')
    out[f'{prefix}_depth_ge1m'] = (out[f'{prefix}_depth_max_m'] >= 1.0).astype('int8')
    out[f'{prefix}_depth_ge3m'] = (out[f'{prefix}_depth_max_m'] >= 3.0).astype('int8')
    out[f'{prefix}_depth_ge5m'] = (out[f'{prefix}_depth_max_m'] >= 5.0).astype('int8')
    out[f'{prefix}_depth_ge10m'] = (out[f'{prefix}_depth_max_m'] >= 10.0).astype('int8')

    return out


In [91]:
# 津波（A40_003）
train_tsunami = add_inundation_polygon_features(
    prop_df=train_df_geo,
    poly_gdf=tsunami_df,
    depth_col='A40_003',
    prefix='tsunami',
    prop_input_crs='EPSG:4326',   # lon/lat の実態に合わせて調整（後述）
    poly_input_crs='EPSG:6668',   # tsunami_df が EPSG:6668
    work_crs='EPSG:6677',
    max_dist_m=10000.0
)

test_tsunami = add_inundation_polygon_features(
    prop_df=test_df_geo,
    poly_gdf=tsunami_df,
    depth_col='A40_003',
    prefix='tsunami',
    prop_input_crs='EPSG:4326',
    poly_input_crs='EPSG:6668',
    work_crs='EPSG:6677',
    max_dist_m=10000.0
)

# 高潮（A49_003）
train_takashio = add_inundation_polygon_features(
    prop_df=train_df_geo,
    poly_gdf=takashio_df,
    depth_col='A49_003',
    prefix='takashio',
    prop_input_crs='EPSG:4326',
    poly_input_crs='EPSG:6668',
    work_crs='EPSG:6677',
    max_dist_m=10000.0
)

test_takashio = add_inundation_polygon_features(
    prop_df=test_df_geo,
    poly_gdf=takashio_df,
    depth_col='A49_003',
    prefix='takashio',
    prop_input_crs='EPSG:4326',
    poly_input_crs='EPSG:6668',
    work_crs='EPSG:6677',
    max_dist_m=10000.0
)

# 追加
train_df_geo = pd.concat([train_df_geo, train_tsunami, train_takashio], axis=1)
test_df_geo  = pd.concat([test_df_geo,  test_tsunami,  test_takashio], axis=1)


In [92]:
print('tsunami_in hit rate train:', train_df_geo['tsunami_in'].mean())
print('tsunami_in hit rate test :', test_df_geo['tsunami_in'].mean())

print('takashio_in hit rate train:', train_df_geo['takashio_in'].mean())
print('takashio_in hit rate test :', test_df_geo['takashio_in'].mean())

tsunami_in hit rate train: 0.055448390323254305
tsunami_in hit rate test : 0.05870843227763103
takashio_in hit rate train: 0.15667007397148855
takashio_in hit rate test : 0.1619395750509174


In [93]:
def postprocess_inund_features(df: pd.DataFrame, prefix: str, max_dist_m: float) -> pd.DataFrame:
    out = df.copy()

    dist = f'{prefix}_dist_m'
    if dist in out.columns:
        out[dist] = out[dist].astype('float32').fillna(np.float32(max_dist_m + 1.0))
        out[f'{prefix}_dist_log1p'] = np.log1p(out[dist].clip(upper=10000)).astype('float32')
        out[f'{prefix}_near_500m'] = (out[dist] <= 500.0).astype('int8')
        out[f'{prefix}_near_1km'] = (out[dist] <= 1000.0).astype('int8')
        out[f'{prefix}_near_3km'] = (out[dist] <= 3000.0).astype('int8')

    return out

In [94]:
train_df_geo = postprocess_inund_features(train_df_geo, 'tsunami', max_dist_m=10000.0)
train_df_geo = postprocess_inund_features(train_df_geo, 'takashio', max_dist_m=10000.0)
test_df_geo = postprocess_inund_features(test_df_geo, 'tsunami', max_dist_m=10000.0)
test_df_geo = postprocess_inund_features(test_df_geo, 'takashio', max_dist_m=10000.0)

## 避難施設

In [95]:
hinanjo_path = gis_path + '避難施設/hinanjo.parquet'

In [96]:
hinanjo_df = gpd.read_parquet(hinanjo_path)

In [97]:
from __future__ import annotations

import numpy as np
import pandas as pd
import geopandas as gpd

def add_hinanjo_features(
    prop_df: pd.DataFrame,
    hinanjo_gdf: gpd.GeoDataFrame,
    lon_col: str = 'lon_jgd',
    lat_col: str = 'lat_jgd',
    prefix: str = 'hinanjo',
    prop_src_crs: str = 'EPSG:4326',
    hinanjo_src_crs: str = 'EPSG:4612',
    work_crs_m: str = 'EPSG:6677',
    max_dist_m: float = 10000.0,
    radii_m: tuple[int, ...] = (500, 1000, 2000),
    hazard_flags: tuple[str, ...] = ('for_jishin_flg', 'for_tsunami_flg', 'for_suigai_flg', 'for_kazan_flg', 'for_all_flg'),
    use_top_shubetsu_k: int = 2,
    chunk_size: int = 50000,
) -> pd.DataFrame:
    """
    避難所(ポイント)から特徴量を作る（GeoPandas sjoin(distance=) 不要）。
    - 最近傍距離 + clip + log1p
    - 半径内: count, capacity sum/max, area sum/max (+log1p)
    - 災害フラグ別: 最近傍距離 + (r=1000,2000 の count/cap sum)
    - shubetsu上位: 最近傍距離(+log1p) + 1000m以内count/cap sum
    """

    # scipy が無い環境もあるので、import を関数内にしてエラーを明確化
    try:
        from scipy.spatial import cKDTree
    except Exception as e:
        raise ImportError('scipy が必要です（cKDTree を使います）。pip install scipy をお願いします。') from e

    out = pd.DataFrame(index=prop_df.index)

    # --- 物件点 -> メートルCRS ---
    prop_g = gpd.GeoDataFrame(
        prop_df[[lon_col, lat_col]].copy(),
        geometry=gpd.points_from_xy(prop_df[lon_col], prop_df[lat_col]),
        crs=prop_src_crs,
        index=prop_df.index,
    ).to_crs(work_crs_m)

    # --- 避難所 -> メートルCRS ---
    h = hinanjo_gdf.copy()
    if h.crs is None:
        h = h.set_crs(hinanjo_src_crs)
    h = h.to_crs(work_crs_m)

    # 数値整形（-1 等は欠損扱い）
    if 'capacity' in h.columns:
        h['capacity'] = pd.to_numeric(h['capacity'], errors='coerce')
        h.loc[h['capacity'] < 0, 'capacity'] = np.nan
    else:
        h['capacity'] = np.nan

    if 'area' in h.columns:
        h['area'] = pd.to_numeric(h['area'], errors='coerce')
        h.loc[h['area'] < 0, 'area'] = np.nan
    else:
        h['area'] = np.nan

    if 'shubetsu' not in h.columns:
        h['shubetsu'] = pd.Series(index=h.index, dtype='string')

    # 座標配列
    prop_xy = np.column_stack([prop_g.geometry.x.values, prop_g.geometry.y.values]).astype('float64')
    h_xy = np.column_stack([h.geometry.x.values, h.geometry.y.values]).astype('float64')

    # KDTree 構築（全避難所）
    tree_all = cKDTree(h_xy)

    # ========== 1) 最近傍距離（全避難所） ==========
    d1, _ = tree_all.query(prop_xy, k=1, distance_upper_bound=max_dist_m)
    # cKDTree は範囲外を inf にする
    d1 = np.where(np.isfinite(d1), d1, np.nan).astype('float32')

    out[f'{prefix}_dist_m'] = d1
    out[f'{prefix}_dist_m_clip'] = np.clip(d1, 0.0, float(max_dist_m)).astype('float32')
    out[f'{prefix}_dist_log1p'] = np.log1p(np.nan_to_num(out[f'{prefix}_dist_m_clip'].values, nan=float(max_dist_m))).astype('float32')

    # ========== 2) 半径内集計（全避難所） ==========
    cap = h['capacity'].values.astype('float32')
    area = h['area'].values.astype('float32')

    for r in radii_m:
        cnt = np.zeros(len(prop_xy), dtype='int32')
        cap_sum = np.full(len(prop_xy), np.nan, dtype='float32')
        cap_max = np.full(len(prop_xy), np.nan, dtype='float32')
        area_sum = np.full(len(prop_xy), np.nan, dtype='float32')
        area_max = np.full(len(prop_xy), np.nan, dtype='float32')

        for st in range(0, len(prop_xy), chunk_size):
            ed = min(st + chunk_size, len(prop_xy))
            neigh = tree_all.query_ball_point(prop_xy[st:ed], r=r)

            for i, idxs in enumerate(neigh):
                if not idxs:
                    continue
                cnt[st + i] = len(idxs)

                c = cap[idxs]
                a = area[idxs]

                # sum は min_count=1 相当（全欠損なら NaN）
                if np.isfinite(c).any():
                    cap_sum[st + i] = np.nansum(c)
                    cap_max[st + i] = np.nanmax(c)
                if np.isfinite(a).any():
                    area_sum[st + i] = np.nansum(a)
                    area_max[st + i] = np.nanmax(a)

        out[f'{prefix}_cnt_r{r}'] = cnt.astype('int16')
        out[f'{prefix}_cap_sum_r{r}'] = cap_sum
        out[f'{prefix}_cap_max_r{r}'] = cap_max
        out[f'{prefix}_area_sum_r{r}'] = area_sum
        out[f'{prefix}_area_max_r{r}'] = area_max

        out[f'{prefix}_cap_sum_log1p_r{r}'] = np.log1p(np.nan_to_num(cap_sum, nan=0.0)).astype('float32')
        out[f'{prefix}_area_sum_log1p_r{r}'] = np.log1p(np.nan_to_num(area_sum, nan=0.0)).astype('float32')

    # ========== 3) 災害フラグ別 ==========
    for hf in hazard_flags:
        if hf not in h.columns:
            continue

        mask = h[hf].fillna(0).astype(int).values == 1
        if mask.sum() == 0:
            out[f'{prefix}_dist_{hf}_m'] = np.float32(np.nan)
            out[f'{prefix}_dist_{hf}_clip'] = np.float32(np.nan)
            out[f'{prefix}_dist_{hf}_log1p'] = np.float32(np.nan)
            for r in (1000, 2000):
                if r in radii_m:
                    out[f'{prefix}_cnt_{hf}_r{r}'] = np.int16(0)
                    out[f'{prefix}_cap_sum_{hf}_r{r}'] = np.float32(np.nan)
            continue

        h_xy_h = h_xy[mask]
        tree_h = cKDTree(h_xy_h)
        cap_h = cap[mask]

        d_h, _ = tree_h.query(prop_xy, k=1, distance_upper_bound=max_dist_m)
        d_h = np.where(np.isfinite(d_h), d_h, np.nan).astype('float32')

        out[f'{prefix}_dist_{hf}_m'] = d_h
        out[f'{prefix}_dist_{hf}_clip'] = np.clip(d_h, 0.0, float(max_dist_m)).astype('float32')
        out[f'{prefix}_dist_{hf}_log1p'] = np.log1p(np.nan_to_num(out[f'{prefix}_dist_{hf}_clip'].values, nan=float(max_dist_m))).astype('float32')

        for r in (1000, 2000):
            if r not in radii_m:
                continue

            cnt = np.zeros(len(prop_xy), dtype='int32')
            cap_sum = np.full(len(prop_xy), np.nan, dtype='float32')

            for st in range(0, len(prop_xy), chunk_size):
                ed = min(st + chunk_size, len(prop_xy))
                neigh = tree_h.query_ball_point(prop_xy[st:ed], r=r)
                for i, idxs in enumerate(neigh):
                    if not idxs:
                        continue
                    cnt[st + i] = len(idxs)
                    c = cap_h[idxs]
                    if np.isfinite(c).any():
                        cap_sum[st + i] = np.nansum(c)

            out[f'{prefix}_cnt_{hf}_r{r}'] = cnt.astype('int16')
            out[f'{prefix}_cap_sum_{hf}_r{r}'] = cap_sum

    # ========== 4) 種別(shubetsu)上位 ==========
    if use_top_shubetsu_k and use_top_shubetsu_k > 0:
        vc = h['shubetsu'].astype('string').value_counts(dropna=True)
        top_vals = vc.head(use_top_shubetsu_k).index.tolist()

        for v in top_vals:
            m = h['shubetsu'].astype('string') == v
            if m.sum() == 0:
                continue
            hv_xy = h_xy[m.values]
            tree_v = cKDTree(hv_xy)
            cap_v = cap[m.values]

            name = str(v).replace(' ', '_')
            d_v, _ = tree_v.query(prop_xy, k=1, distance_upper_bound=max_dist_m)
            d_v = np.where(np.isfinite(d_v), d_v, np.nan).astype('float32')

            out[f'{prefix}_dist_shubetsu_{name}_m'] = d_v
            out[f'{prefix}_dist_shubetsu_{name}_log1p'] = np.log1p(np.clip(np.nan_to_num(d_v, nan=float(max_dist_m)), 0.0, float(max_dist_m))).astype('float32')

            # 軽量に 1000m だけ
            r = 1000
            if r in radii_m:
                cnt = np.zeros(len(prop_xy), dtype='int32')
                cap_sum = np.full(len(prop_xy), np.nan, dtype='float32')
                for st in range(0, len(prop_xy), chunk_size):
                    ed = min(st + chunk_size, len(prop_xy))
                    neigh = tree_v.query_ball_point(prop_xy[st:ed], r=r)
                    for i, idxs in enumerate(neigh):
                        if not idxs:
                            continue
                        cnt[st + i] = len(idxs)
                        c = cap_v[idxs]
                        if np.isfinite(c).any():
                            cap_sum[st + i] = np.nansum(c)

                out[f'{prefix}_cnt_shubetsu_{name}_r{r}'] = cnt.astype('int16')
                out[f'{prefix}_cap_sum_shubetsu_{name}_r{r}'] = cap_sum

    # 型揃え
    for c in out.columns:
        if c.endswith('_cnt_r500') or c.endswith('_cnt_r1000') or c.endswith('_cnt_r2000') or '_cnt_' in c:
            out[c] = out[c].astype('int16', errors='ignore')
    for c in out.columns:
        if out[c].dtype == 'float64':
            out[c] = out[c].astype('float32')

    return out

In [98]:
train_hinanjo = add_hinanjo_features(train_df_geo, hinanjo_df, prop_src_crs='EPSG:4326', prefix='hinanjo')
test_hinanjo  = add_hinanjo_features(test_df_geo,  hinanjo_df, prop_src_crs='EPSG:4326', prefix='hinanjo')

train_df_geo = pd.concat([train_df_geo, train_hinanjo], axis=1)
test_df_geo  = pd.concat([test_df_geo,  test_hinanjo], axis=1)

## 特徴量の追加・削除

In [99]:
train_df_geo.columns.to_list()

['target_ym',
 'building_id',
 'unit_id',
 'lat',
 'lon',
 'target_year',
 'lon_jgd',
 'lat_jgd',
 'nearest_land_price',
 'distance_to_landpoint_m',
 'log_land_price',
 'weighted_land_price_3',
 'log_weighted_land_price_3',
 'nearest_land_price_prev',
 'weighted_land_price_3_prev',
 'land_price_yoy_nearest',
 'land_price_yoy_w3',
 'land_price_dlog_nearest',
 'land_price_dlog_w3',
 'pop_dist_nn_m',
 'PTN_2020_nn',
 'RTA_2025_nn',
 'RTB_2025_nn',
 'RTC_2025_nn',
 'RTD_2025_nn',
 'RTE_2025_nn',
 'pop_trend_rate_nn',
 'road_len_total',
 'road_len_wide',
 'road_len_narrow',
 'road_wide_ratio',
 'road_narrow_ratio',
 'road_len_total_gap',
 'road_narrow_ratio_gap',
 'road_len_density',
 'road_len_density_gap',
 'dist_to_road_any_m',
 'dist_to_road_major_m',
 'dist_to_road_highway_m',
 'road_cnt_any_in_100m',
 'road_cnt_major_in_100m',
 'road_cnt_any_in_300m',
 'road_cnt_major_in_300m',
 'road_cnt_any_in_500m',
 'road_cnt_major_in_500m',
 'A29_005',
 'zone_group',
 'zone_residential_rank',
 'i

In [100]:
hinanjo_df[['for_all_flg', 'for_jishin_flg', 'for_tsunami_flg', 'for_suigai_flg', 'for_kazan_flg']].value_counts()

for_all_flg  for_jishin_flg  for_tsunami_flg  for_suigai_flg  for_kazan_flg
1            0               0                0               0                101967
0            1               0                1               0                  6856
             0               0                1               0                  5481
                             1                0               0                  4041
             1               0                0               0                  4036
                             1                1               0                  1569
             0               1                1               0                   504
             1               1                0               0                   441
             0               0                0               0                   256
             1               1                1               1                   250
             0               1                1               1 

In [103]:
add_cols = [
    # 地価
    'nearest_land_price', 'weighted_land_price_3', 'distance_to_landpoint_m', 'log_land_price', 'log_weighted_land_price_3',
    'land_price_yoy_nearest', 'land_price_yoy_w3', 'land_price_dlog_nearest', 'land_price_dlog_w3',

    # 将来人口
    'PTN_2020_nn', 'RTA_2025_nn', 'RTB_2025_nn', 'RTC_2025_nn', 'RTD_2025_nn', 'RTE_2025_nn',
    'pop_trend_rate_nn',

    # 道路
    'road_len_total', 'road_len_wide', 'road_len_narrow',
    'road_wide_ratio', 'road_narrow_ratio',
    'road_len_total_gap', 'road_narrow_ratio_gap',
    'road_len_density', 'road_len_density_gap',
    'dist_to_road_any_m', 'dist_to_road_major_m', 'dist_to_road_highway_m',
    'road_cnt_any_in_100m', 'road_cnt_major_in_100m',
    'road_cnt_any_in_300m', 'road_cnt_major_in_300m',
    'road_cnt_any_in_500m', 'road_cnt_major_in_500m',

    # 用途地域
    'zone_group', 'zone_residential_rank',
    'is_lowrise_residential', 'kenpei', 'youseki',

    # 都市計画
    'is_urbanized_area', 'is_urban_control_area', 'is_fireproof_area',
    'is_quasi_fireproof_area', 'has_height_limit', 'has_district_plan',
    'is_land_readjustment_area', 'is_high_utilization_area',
    'is_urban_renaissance_area', 'is_special_far_area',
    'is_highrise_residential_area', 'is_disaster_prevention_block',
    'is_redevelopment_core_area',

    # 標高
    'elev_mean', 'elev_range', 'slope_mean', 'slope_max', 'slope_range',
    'station_power_sum3', 'station_power_max3',

    # 災害危険区域
    'disaster_hit', 'disaster_n_types', 'disaster_score_sum', 'disaster_score_max', 'disaster_score',

    # 学校
    'elem_school_500m', 'junior_school_1km', 'dist_elem_school_m', 'dist_junior_school_m',

    # 病院
    'dist_hospital_m', 'dist_clinic_m',
    'hospital_1km', 'dist_hospital_log', 'clinic_500m', 'dist_clinic_log',
    'hospital_beds_nearest', 'hospital_is_emergency_nearest', 'hospital_is_disaster_nearest',
    
    # 洪水浸水（内水）
    'ame_dist_log', 'ame_depth_max_log1p', 'ame_depth_effect',
    
    # 洪水浸水（外水）
    'kouzui_keikaku_in', 'kouzui_keikaku_rank_max', 'kouzui_keikaku_dist_m_log1p',
    'kouzui_saidai_in', 'kouzui_saidai_rank_max', 'kouzui_saidai_dist_m_log1p',
    'kouzui_keikaku_rank_ge3', 'kouzui_saidai_rank_ge3',
    
    # 土砂災害
    'kyuusha_in', 'kyuusha_dist_m', 
    'keikai_genshou1_in', 'keikai_genshou2_in', 'keikai_genshou3_in', 'keikai_kuiki1_in', 'keikai_kuiki2_in', 'keikai_kuiki3_in', 'keikai_kuiki4_in',
    'keikai_kouji_date_min', 'keikai_dist_m',
    'dosham_in', 'dosham_phen_gake', 'dosham_phen_doseki', 'dosham_phen_jisuberi', 'dosham_phen_nadare',
    'dosham_A30a5_005_max', 'dosham_A30a5_006_max', 'dosham_A30a5_007_max', 'dosham_A30a5_008_max', 'dosham_A30a5_009_max', 'dosham_A30a5_010_max',
    
    # 地盤沈下
    # 'jiban_dist_m_log1p', 'jiban_near_100m', 'jiban_near_300m' 'jiban_near_1000m',
    # 'jiban_year_nearest', 'jiban_year_age',
    # 'jiban_type_02_nearest', 'jiban_type_04_nearest' ,'jiban_type_70_nearest' ,'jiban_type_80_nearest' ,'jiban_type_90_nearest', 'jiban_type_99_nearest',

    # 沿岸浸水
    'tsunami_in', 'tsunami_depth_max_m', 'tsunami_depth_rank_max', 'tsunami_dist_log1p',
    'tsunami_depth_max_log1p', 'tsunami_depth_ge1m', 'tsunami_depth_ge3m', 'tsunami_depth_ge5m', 'tsunami_depth_ge10m',
    'takashio_in', 'takashio_depth_max_m', 'takashio_depth_rank_max', 'takashio_dist_log1p', 
    'tsunami_near_500m', 'tsunami_near_1km', 'tsunami_near_3km',
    'takashio_depth_max_log1p', 'takashio_depth_ge1m', 'takashio_depth_ge3m', 'takashio_depth_ge5m', 'takashio_depth_ge10m',
    'takashio_near_500m', 'takashio_near_1km', 'takashio_near_3km',

    # 避難所
    'hinanjo_dist_m', 'hinanjo_dist_m_clip', 'hinanjo_dist_log1p', 'hinanjo_cnt_r500', 'hinanjo_cap_sum_r500',
    'hinanjo_cap_max_r500', 'hinanjo_area_sum_r500', 'hinanjo_area_max_r500', 'hinanjo_cap_sum_log1p_r500',
    'hinanjo_area_sum_log1p_r500', 'hinanjo_cnt_r1000', 'hinanjo_cap_sum_r1000', 'hinanjo_cap_max_r1000',
    'hinanjo_area_sum_r1000', 'hinanjo_area_max_r1000', 'hinanjo_cap_sum_log1p_r1000', 'hinanjo_area_sum_log1p_r1000',
    'hinanjo_cnt_r2000', 'hinanjo_cap_sum_r2000', 'hinanjo_cap_max_r2000', 'hinanjo_area_sum_r2000',
    'hinanjo_area_max_r2000', 'hinanjo_cap_sum_log1p_r2000', 'hinanjo_area_sum_log1p_r2000',
    'hinanjo_dist_for_jishin_flg_m', 'hinanjo_dist_for_jishin_flg_clip', 'hinanjo_dist_for_jishin_flg_log1p',
    'hinanjo_cnt_for_jishin_flg_r1000', 'hinanjo_cap_sum_for_jishin_flg_r1000', 
    'hinanjo_cnt_for_jishin_flg_r2000', 'hinanjo_cap_sum_for_jishin_flg_r2000',
    'hinanjo_dist_for_tsunami_flg_m', 'hinanjo_dist_for_tsunami_flg_clip',
    'hinanjo_dist_for_tsunami_flg_log1p', 'hinanjo_cnt_for_tsunami_flg_r1000',
    'hinanjo_cap_sum_for_tsunami_flg_r1000', 'hinanjo_cnt_for_tsunami_flg_r2000',
    'hinanjo_cap_sum_for_tsunami_flg_r2000', 'hinanjo_dist_for_suigai_flg_m',
    'hinanjo_dist_for_suigai_flg_clip', 'hinanjo_dist_for_suigai_flg_log1p',
    'hinanjo_cnt_for_suigai_flg_r1000', 'hinanjo_cap_sum_for_suigai_flg_r1000',
    'hinanjo_cnt_for_suigai_flg_r2000', 'hinanjo_cap_sum_for_suigai_flg_r2000',
    'hinanjo_dist_for_kazan_flg_m', 'hinanjo_dist_for_kazan_flg_clip',
    'hinanjo_dist_for_kazan_flg_log1p', 'hinanjo_cnt_for_kazan_flg_r1000',
    'hinanjo_cap_sum_for_kazan_flg_r1000', 'hinanjo_cnt_for_kazan_flg_r2000',
    'hinanjo_cap_sum_for_kazan_flg_r2000', 'hinanjo_dist_for_all_flg_m',
    'hinanjo_dist_for_all_flg_clip', 'hinanjo_dist_for_all_flg_log1p',
    'hinanjo_cnt_for_all_flg_r1000', 'hinanjo_cap_sum_for_all_flg_r1000',
    'hinanjo_cnt_for_all_flg_r2000', 'hinanjo_cap_sum_for_all_flg_r2000',
    'hinanjo_dist_shubetsu_避難所_m', 'hinanjo_dist_shubetsu_避難所_log1p',
    'hinanjo_cnt_shubetsu_避難所_r1000', 'hinanjo_cap_sum_shubetsu_避難所_r1000',
    'hinanjo_dist_shubetsu_避難場所_m', 'hinanjo_dist_shubetsu_避難場所_log1p',
    'hinanjo_cnt_shubetsu_避難場所_r1000', 'hinanjo_cap_sum_shubetsu_避難場所_r1000'

]

## 出力

In [104]:
train_df_geo[pkey_cols + add_cols].to_parquet(f'{intermediate_path}train_df_geo_v{geo_ver}.parquet')
test_df_geo[pkey_cols + add_cols].to_parquet(f'{intermediate_path}test_df_geo_v{geo_ver}.parquet')